In [ ]:
## This file implements neural networks and logistic regression on p11presabsSTCC_quant.
## Due to the imbalanced dataset, we implement the over-sampling method and the combination of over- and under-sampling
## method.
## For fully-connected neural networks, the accuracy is 92.08% for combination data, and 87.02% for over-sampling data 
## after improvements and regularization.
## To improve accuracy, we use lasso to select important features and thus have new training and test datasets.
## However, with the new datasets, the accuracy becomes 93.94% for combination data and 83.21% for over-sampling data.
## For logistic regression, the accuracies are 97.03% for combination data and 90.84% for over-sampling data.
## For random forest, the accuracy is 98.02% for combination and 96.95% for over-sampling data.
## For random forest with cross-validation, the mean accuracy is 98.27% for combination data, and 94.76% for over-sampling.

In [1]:
import pandas as pd

df = pd.read_csv('/Users/Rebecca/Desktop/Claudia/neural network/phage_quant/p11presabsSTCC_quant.csv')
df.shape

(255, 976)

In [2]:
df.rename(columns={'Unnamed: 0':'id'}, inplace=True)

In [3]:
df['pheno']

0      0.971625
1      0.591125
2      0.902750
3      1.169875
4      1.147000
5      1.112875
6      0.936500
7      0.707000
8      0.806375
9      0.921500
10     0.510000
11     0.958000
12     1.036750
13     0.427625
14     1.052250
15     1.079500
16     0.390375
17     0.385125
18     1.025500
19     0.225750
20     0.295875
21     0.348250
22     0.507375
23     1.114000
24     1.089625
25     0.163000
26     0.888125
27     0.198375
28     0.828250
29     1.153375
         ...   
225    0.975300
226    0.698300
227    0.933700
228    1.012100
229    1.104800
230    0.621000
231    0.941250
232    0.928500
233    0.902250
234    0.897000
235    1.047375
236    0.992625
237    0.901250
238    1.075750
239    0.758000
240    0.831750
241    0.891000
242    0.369125
243    0.772000
244    0.986750
245    0.975375
246    1.125250
247    0.828750
248    1.086250
249    0.854750
250    0.905000
251    0.572500
252    1.071000
253    1.000500
254    0.955125
Name: pheno, Length: 255

In [4]:
df.head()

,id,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno,ST,CC
0,107,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.971625,5,5
1,109,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.591125,8,8
2,115,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0.902750,5,5
3,120335,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.169875,5,5
4,120337,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1.147000,5,5


In [5]:
if 0.5 in df['pheno']:
    print: "0.5 is in the list"

In [6]:
df['pheno'] = [1 if i>0.5 else 0 for i in df['pheno']] # convert pheno into binary

In [7]:
df['pheno']
df['pheno'].value_counts()

1    218
0     37
Name: pheno, dtype: int64

In [8]:
df.shape

(255, 976)

In [9]:
df_clean = df.drop(columns=['id'])

In [10]:
df_clean.shape

(255, 975)

In [11]:
df_clean.head()

,TTTTTTGTAATTTT,TTTTTTGTAATTTTT,TTTTTTATTTTGGAT,TTTTTTATTTTGGATAA,TTTTTTATTTTGGATAAAAGGAG,TTTTTTAGTCGTTTTT,TTTTTATCGTTTACT,TTTTTAGTCGTTTTT,TTTTTAGTCGTTTTTT,TTTTTAGGTAAGG,...,group_1300,group_1972,group_2563,group_2881,group_7076,group_8027,group_892,pheno,ST,CC
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
1,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,8,8
2,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
3,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5
4,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,1,5,5


In [12]:
X = df_clean.loc[:, df_clean.columns != 'pheno'].values
y = df_clean['pheno'].values
print(X.shape, y.shape)

(255, 974) (255,)


In [13]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_comb, y_comb = smote_enn.fit_resample(X, y)
print(sorted(Counter(y_comb).items()))

Using TensorFlow backend.


[(0, 175), (1, 161)]


In [14]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_over, y_over = overS.fit_resample(X, y)
print(sorted(Counter(y_over).items()))

[(0, 218), (1, 218)]


In [15]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_train_comb, X_test_comb, y_train_comb, y_test_comb = train_test_split(X_comb, y_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_comb)

In [16]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_train_over, X_test_over, y_train_over, y_test_over = train_test_split(X_over, y_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_over)

In [17]:
############# Fully-Connected Neural Network ################

In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.regularizers import l1
import numpy as np

In [19]:
#### neural network on combination data
model1_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [20]:
model1_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [21]:
model1_comb.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 235 samples, validate on 101 samples
Epoch 1/1000
235/235 [==============================] - 0s 514us/step - loss: 5.7130 - accuracy: 0.5489 - val_loss: 0.6223 - val_accuracy: 0.8119
Epoch 2/1000
235/235 [==============================] - 0s 100us/step - loss: 0.5994 - accuracy: 0.7745 - val_loss: 0.5940 - val_accuracy: 0.8317
Epoch 3/1000
235/235 [==============================] - 0s 129us/step - loss: 0.5598 - accuracy: 0.8681 - val_loss: 0.5908 - val_accuracy: 0.8614
Epoch 4/1000
235/235 [==============================] - 0s 131us/step - loss: 0.5306 - accuracy: 0.8298 - val_loss: 0.5377 - val_accuracy: 0.8317
Epoch 5/1000
235/235 [==============================] - 0s 135us/step - loss: 0.5052 - accuracy: 0.8723 - val_loss: 0.5063 - val_accuracy: 0.7921
Epoch 6/1000
235/235 [==============================] - 0s 128us/step - loss: 0.6937 - accuracy: 0.8383 - val_loss: 0.5161 - val_accuracy: 0.8713
Epoch 7/1000
235/235 [==============================] - 0s 131us/step - loss: 

235/235 [==============================] - 0s 189us/step - loss: 0.2348 - accuracy: 0.9277 - val_loss: 0.3171 - val_accuracy: 0.8911
Epoch 57/1000
235/235 [==============================] - 0s 185us/step - loss: 0.2311 - accuracy: 0.9277 - val_loss: 0.3151 - val_accuracy: 0.8911
Epoch 58/1000
235/235 [==============================] - 0s 197us/step - loss: 0.2291 - accuracy: 0.9277 - val_loss: 0.3141 - val_accuracy: 0.8911
Epoch 59/1000
235/235 [==============================] - 0s 145us/step - loss: 0.2279 - accuracy: 0.9277 - val_loss: 0.3128 - val_accuracy: 0.8911
Epoch 60/1000
235/235 [==============================] - 0s 157us/step - loss: 0.2280 - accuracy: 0.9277 - val_loss: 0.3116 - val_accuracy: 0.8911
Epoch 61/1000
235/235 [==============================] - 0s 141us/step - loss: 0.2265 - accuracy: 0.9234 - val_loss: 0.3100 - val_accuracy: 0.8911
Epoch 62/1000
235/235 [==============================] - 0s 108us/step - loss: 0.2259 - accuracy: 0.9319 - val_loss: 0.3190 - val_ac

Epoch 112/1000
235/235 [==============================] - 0s 115us/step - loss: 0.1951 - accuracy: 0.9404 - val_loss: 0.2596 - val_accuracy: 0.9307
Epoch 113/1000
235/235 [==============================] - 0s 107us/step - loss: 0.1942 - accuracy: 0.9404 - val_loss: 0.2592 - val_accuracy: 0.9307
Epoch 114/1000
235/235 [==============================] - 0s 134us/step - loss: 0.1920 - accuracy: 0.9404 - val_loss: 0.2623 - val_accuracy: 0.9109
Epoch 115/1000
235/235 [==============================] - 0s 165us/step - loss: 0.1937 - accuracy: 0.9362 - val_loss: 0.2578 - val_accuracy: 0.9307
Epoch 116/1000
235/235 [==============================] - 0s 191us/step - loss: 0.1904 - accuracy: 0.9404 - val_loss: 0.2584 - val_accuracy: 0.9208
Epoch 117/1000
235/235 [==============================] - 0s 227us/step - loss: 0.1890 - accuracy: 0.9362 - val_loss: 0.2624 - val_accuracy: 0.9010
Epoch 118/1000
235/235 [==============================] - 0s 188us/step - loss: 0.1891 - accuracy: 0.9404 - val_

235/235 [==============================] - 0s 107us/step - loss: 0.1664 - accuracy: 0.9404 - val_loss: 0.2307 - val_accuracy: 0.9208
Epoch 168/1000
235/235 [==============================] - 0s 99us/step - loss: 0.1657 - accuracy: 0.9404 - val_loss: 0.2292 - val_accuracy: 0.9208
Epoch 169/1000
235/235 [==============================] - 0s 102us/step - loss: 0.1679 - accuracy: 0.9362 - val_loss: 0.2294 - val_accuracy: 0.9307
Epoch 170/1000
235/235 [==============================] - 0s 103us/step - loss: 0.1653 - accuracy: 0.9404 - val_loss: 0.2282 - val_accuracy: 0.9307
Epoch 171/1000
235/235 [==============================] - 0s 110us/step - loss: 0.1656 - accuracy: 0.9404 - val_loss: 0.2290 - val_accuracy: 0.9208
Epoch 172/1000
235/235 [==============================] - 0s 96us/step - loss: 0.1638 - accuracy: 0.9404 - val_loss: 0.2456 - val_accuracy: 0.8911
Epoch 173/1000
235/235 [==============================] - 0s 111us/step - loss: 0.1698 - accuracy: 0.9319 - val_loss: 0.2270 - va

235/235 [==============================] - 0s 94us/step - loss: 0.1474 - accuracy: 0.9447 - val_loss: 0.2103 - val_accuracy: 0.9307
Epoch 223/1000
235/235 [==============================] - 0s 125us/step - loss: 0.1492 - accuracy: 0.9447 - val_loss: 0.2121 - val_accuracy: 0.9208
Epoch 224/1000
235/235 [==============================] - 0s 107us/step - loss: 0.1464 - accuracy: 0.9447 - val_loss: 0.2230 - val_accuracy: 0.9307
Epoch 225/1000
235/235 [==============================] - 0s 101us/step - loss: 0.1509 - accuracy: 0.9447 - val_loss: 0.2088 - val_accuracy: 0.9208
Epoch 226/1000
235/235 [==============================] - 0s 99us/step - loss: 0.1462 - accuracy: 0.9447 - val_loss: 0.2099 - val_accuracy: 0.9307
Epoch 227/1000
235/235 [==============================] - 0s 103us/step - loss: 0.1471 - accuracy: 0.9447 - val_loss: 0.2080 - val_accuracy: 0.9208
Epoch 228/1000
235/235 [==============================] - 0s 110us/step - loss: 0.1446 - accuracy: 0.9447 - val_loss: 0.2103 - va

...

235/235 [==============================] - 0s 107us/step - loss: 0.0989 - accuracy: 0.9617 - val_loss: 0.1638 - val_accuracy: 0.9307
Epoch 773/1000
235/235 [==============================] - 0s 122us/step - loss: 0.0990 - accuracy: 0.9617 - val_loss: 0.1599 - val_accuracy: 0.9406
Epoch 774/1000
235/235 [==============================] - 0s 100us/step - loss: 0.0988 - accuracy: 0.9617 - val_loss: 0.1586 - val_accuracy: 0.9406
Epoch 775/1000
235/235 [==============================] - 0s 107us/step - loss: 0.0990 - accuracy: 0.9617 - val_loss: 0.1588 - val_accuracy: 0.9406
Epoch 776/1000
235/235 [==============================] - 0s 106us/step - loss: 0.0989 - accuracy: 0.9617 - val_loss: 0.1568 - val_accuracy: 0.9406
Epoch 777/1000
235/235 [==============================] - 0s 153us/step - loss: 0.0990 - accuracy: 0.9617 - val_loss: 0.1569 - val_accuracy: 0.9406
Epoch 778/1000
235/235 [==============================] - 0s 115us/step - loss: 0.0990 - accuracy: 0.9617 - val_loss: 0.1582 - 

235/235 [==============================] - 0s 88us/step - loss: 0.0984 - accuracy: 0.9617 - val_loss: 0.1608 - val_accuracy: 0.9406
Epoch 828/1000
235/235 [==============================] - 0s 110us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1576 - val_accuracy: 0.9406
Epoch 829/1000
235/235 [==============================] - 0s 107us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1580 - val_accuracy: 0.9406
Epoch 830/1000
235/235 [==============================] - 0s 100us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1570 - val_accuracy: 0.9406
Epoch 831/1000
235/235 [==============================] - 0s 102us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1568 - val_accuracy: 0.9406
Epoch 832/1000
235/235 [==============================] - 0s 108us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1571 - val_accuracy: 0.9406
Epoch 833/1000
235/235 [==============================] - 0s 96us/step - loss: 0.0982 - accuracy: 0.9617 - val_loss: 0.1559 - va

235/235 [==============================] - 0s 99us/step - loss: 0.0978 - accuracy: 0.9617 - val_loss: 0.1558 - val_accuracy: 0.9406
Epoch 883/1000
235/235 [==============================] - 0s 121us/step - loss: 0.0977 - accuracy: 0.9617 - val_loss: 0.1569 - val_accuracy: 0.9406
Epoch 884/1000
235/235 [==============================] - 0s 112us/step - loss: 0.0976 - accuracy: 0.9617 - val_loss: 0.1565 - val_accuracy: 0.9406
Epoch 885/1000
235/235 [==============================] - 0s 97us/step - loss: 0.0976 - accuracy: 0.9617 - val_loss: 0.1553 - val_accuracy: 0.9406
Epoch 886/1000
235/235 [==============================] - 0s 108us/step - loss: 0.0978 - accuracy: 0.9617 - val_loss: 0.1563 - val_accuracy: 0.9406
Epoch 887/1000
235/235 [==============================] - 0s 103us/step - loss: 0.0976 - accuracy: 0.9617 - val_loss: 0.1564 - val_accuracy: 0.9406
Epoch 888/1000
235/235 [==============================] - 0s 99us/step - loss: 0.0975 - accuracy: 0.9617 - val_loss: 0.1561 - val

235/235 [==============================] - 0s 105us/step - loss: 0.0971 - accuracy: 0.9617 - val_loss: 0.1578 - val_accuracy: 0.9406
Epoch 938/1000
235/235 [==============================] - 0s 105us/step - loss: 0.0971 - accuracy: 0.9617 - val_loss: 0.1612 - val_accuracy: 0.9406
Epoch 939/1000
235/235 [==============================] - 0s 113us/step - loss: 0.0973 - accuracy: 0.9617 - val_loss: 0.1617 - val_accuracy: 0.9307
Epoch 940/1000
235/235 [==============================] - 0s 101us/step - loss: 0.0972 - accuracy: 0.9617 - val_loss: 0.1565 - val_accuracy: 0.9406
Epoch 941/1000
235/235 [==============================] - 0s 102us/step - loss: 0.0971 - accuracy: 0.9617 - val_loss: 0.1588 - val_accuracy: 0.9406
Epoch 942/1000
235/235 [==============================] - 0s 101us/step - loss: 0.0971 - accuracy: 0.9617 - val_loss: 0.1566 - val_accuracy: 0.9406
Epoch 943/1000
235/235 [==============================] - 0s 103us/step - loss: 0.0973 - accuracy: 0.9617 - val_loss: 0.1582 - 

235/235 [==============================] - 0s 106us/step - loss: 0.0968 - accuracy: 0.9617 - val_loss: 0.1549 - val_accuracy: 0.9406
Epoch 993/1000
235/235 [==============================] - 0s 104us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1559 - val_accuracy: 0.9406
Epoch 994/1000
235/235 [==============================] - 0s 113us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1581 - val_accuracy: 0.9406
Epoch 995/1000
235/235 [==============================] - 0s 108us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1555 - val_accuracy: 0.9406
Epoch 996/1000
235/235 [==============================] - 0s 103us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1553 - val_accuracy: 0.9406
Epoch 997/1000
235/235 [==============================] - 0s 116us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1580 - val_accuracy: 0.9406
Epoch 998/1000
235/235 [==============================] - 0s 98us/step - loss: 0.0967 - accuracy: 0.9617 - val_loss: 0.1563 - v

In [22]:
acc_test_comb = model1_comb.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb*100))

101/101 [==============================] - 0s 90us/step
combination test accuracy: 94.06%


In [23]:
#### add regularizor and dropout
model1_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [24]:
model1_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [25]:
model1_comb2.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 235 samples, validate on 101 samples
Epoch 1/1000
235/235 [==============================] - 0s 847us/step - loss: 2.4370 - accuracy: 0.6340 - val_loss: 1.1413 - val_accuracy: 0.8317
Epoch 2/1000
235/235 [==============================] - 0s 111us/step - loss: 2.2399 - accuracy: 0.7319 - val_loss: 0.6438 - val_accuracy: 0.6931
Epoch 3/1000
235/235 [==============================] - 0s 122us/step - loss: 1.6660 - accuracy: 0.8043 - val_loss: 0.6286 - val_accuracy: 0.6535
Epoch 4/1000
235/235 [==============================] - 0s 130us/step - loss: 2.0112 - accuracy: 0.8426 - val_loss: 0.5987 - val_accuracy: 0.7129
Epoch 5/1000
235/235 [==============================] - 0s 128us/step - loss: 1.7442 - accuracy: 0.8298 - val_loss: 0.5793 - val_accuracy: 0.7624
Epoch 6/1000
235/235 [==============================] - 0s 130us/step - loss: 1.8478 - accuracy: 0.8170 - val_loss: 0.5972 - val_accuracy: 0.6634
Epoch 7/1000
235/235 [==============================] - 0s 127us/step - loss: 

235/235 [==============================] - 0s 129us/step - loss: 1.9981 - accuracy: 0.8255 - val_loss: 0.4261 - val_accuracy: 0.9109
Epoch 57/1000
235/235 [==============================] - 0s 160us/step - loss: 1.5833 - accuracy: 0.8638 - val_loss: 0.4232 - val_accuracy: 0.9109
Epoch 58/1000
235/235 [==============================] - 0s 168us/step - loss: 1.4952 - accuracy: 0.8851 - val_loss: 0.4212 - val_accuracy: 0.9109
Epoch 59/1000
235/235 [==============================] - 0s 209us/step - loss: 1.5312 - accuracy: 0.8638 - val_loss: 0.4195 - val_accuracy: 0.9109
Epoch 60/1000
235/235 [==============================] - 0s 147us/step - loss: 1.1303 - accuracy: 0.8936 - val_loss: 0.4153 - val_accuracy: 0.9109
Epoch 61/1000
235/235 [==============================] - 0s 131us/step - loss: 1.6485 - accuracy: 0.8596 - val_loss: 0.4155 - val_accuracy: 0.9109
Epoch 62/1000
235/235 [==============================] - 0s 159us/step - loss: 1.7338 - accuracy: 0.8511 - val_loss: 0.4153 - val_ac

Epoch 112/1000
235/235 [==============================] - 0s 166us/step - loss: 2.1273 - accuracy: 0.8298 - val_loss: 0.3795 - val_accuracy: 0.9109
Epoch 113/1000
235/235 [==============================] - 0s 194us/step - loss: 1.7416 - accuracy: 0.8553 - val_loss: 0.3815 - val_accuracy: 0.9109
Epoch 114/1000
235/235 [==============================] - 0s 190us/step - loss: 1.6195 - accuracy: 0.8596 - val_loss: 0.3701 - val_accuracy: 0.9307
Epoch 115/1000
235/235 [==============================] - 0s 135us/step - loss: 1.3042 - accuracy: 0.8851 - val_loss: 0.3723 - val_accuracy: 0.9109
Epoch 116/1000
235/235 [==============================] - 0s 147us/step - loss: 1.4027 - accuracy: 0.8851 - val_loss: 0.3692 - val_accuracy: 0.9307
Epoch 117/1000
235/235 [==============================] - 0s 136us/step - loss: 1.7514 - accuracy: 0.8468 - val_loss: 0.3847 - val_accuracy: 0.9109
Epoch 118/1000
235/235 [==============================] - 0s 166us/step - loss: 2.0644 - accuracy: 0.8298 - val_

235/235 [==============================] - 0s 107us/step - loss: 1.6709 - accuracy: 0.8596 - val_loss: 0.3650 - val_accuracy: 0.9208
Epoch 168/1000
235/235 [==============================] - 0s 114us/step - loss: 1.4852 - accuracy: 0.8723 - val_loss: 0.3551 - val_accuracy: 0.9208
Epoch 169/1000
235/235 [==============================] - 0s 106us/step - loss: 2.1814 - accuracy: 0.8298 - val_loss: 0.3543 - val_accuracy: 0.9208
Epoch 170/1000
235/235 [==============================] - 0s 127us/step - loss: 1.8596 - accuracy: 0.8468 - val_loss: 0.3600 - val_accuracy: 0.9208
Epoch 171/1000
235/235 [==============================] - 0s 131us/step - loss: 1.9279 - accuracy: 0.8383 - val_loss: 0.3524 - val_accuracy: 0.9208
Epoch 172/1000
235/235 [==============================] - 0s 106us/step - loss: 1.6057 - accuracy: 0.8596 - val_loss: 0.3534 - val_accuracy: 0.9208
Epoch 173/1000
235/235 [==============================] - 0s 125us/step - loss: 1.8537 - accuracy: 0.8511 - val_loss: 0.3612 - 

235/235 [==============================] - 0s 129us/step - loss: 2.0985 - accuracy: 0.8255 - val_loss: 0.3364 - val_accuracy: 0.9208
Epoch 223/1000
235/235 [==============================] - 0s 126us/step - loss: 1.9650 - accuracy: 0.8340 - val_loss: 0.3243 - val_accuracy: 0.9208
Epoch 224/1000
235/235 [==============================] - 0s 129us/step - loss: 1.3088 - accuracy: 0.8809 - val_loss: 0.3207 - val_accuracy: 0.9307
Epoch 225/1000
235/235 [==============================] - 0s 116us/step - loss: 1.8481 - accuracy: 0.8340 - val_loss: 0.3195 - val_accuracy: 0.9307
Epoch 226/1000
235/235 [==============================] - 0s 112us/step - loss: 1.8334 - accuracy: 0.8426 - val_loss: 0.3221 - val_accuracy: 0.9307
Epoch 227/1000
235/235 [==============================] - 0s 112us/step - loss: 1.5029 - accuracy: 0.8681 - val_loss: 0.3345 - val_accuracy: 0.9208
Epoch 228/1000
235/235 [==============================] - 0s 117us/step - loss: 2.0319 - accuracy: 0.8255 - val_loss: 0.3222 - 

...

235/235 [==============================] - 0s 121us/step - loss: 1.4390 - accuracy: 0.8723 - val_loss: 0.3246 - val_accuracy: 0.9109
Epoch 773/1000
235/235 [==============================] - 0s 133us/step - loss: 1.4208 - accuracy: 0.8809 - val_loss: 0.3262 - val_accuracy: 0.9109
Epoch 774/1000
235/235 [==============================] - 0s 115us/step - loss: 1.5750 - accuracy: 0.8596 - val_loss: 0.3219 - val_accuracy: 0.9208
Epoch 775/1000
235/235 [==============================] - 0s 121us/step - loss: 1.2387 - accuracy: 0.8851 - val_loss: 0.3121 - val_accuracy: 0.9208
Epoch 776/1000
235/235 [==============================] - 0s 110us/step - loss: 1.8082 - accuracy: 0.8553 - val_loss: 0.3160 - val_accuracy: 0.9208
Epoch 777/1000
235/235 [==============================] - 0s 121us/step - loss: 2.0055 - accuracy: 0.8426 - val_loss: 0.3200 - val_accuracy: 0.9208
Epoch 778/1000
235/235 [==============================] - 0s 116us/step - loss: 2.0788 - accuracy: 0.8340 - val_loss: 0.3201 - 

235/235 [==============================] - 0s 116us/step - loss: 2.2744 - accuracy: 0.8213 - val_loss: 0.3212 - val_accuracy: 0.9109
Epoch 828/1000
235/235 [==============================] - 0s 134us/step - loss: 1.4546 - accuracy: 0.8638 - val_loss: 0.3210 - val_accuracy: 0.9109
Epoch 829/1000
235/235 [==============================] - 0s 122us/step - loss: 1.6057 - accuracy: 0.8723 - val_loss: 0.3219 - val_accuracy: 0.9109
Epoch 830/1000
235/235 [==============================] - 0s 140us/step - loss: 1.8359 - accuracy: 0.8426 - val_loss: 0.3232 - val_accuracy: 0.9010
Epoch 831/1000
235/235 [==============================] - 0s 130us/step - loss: 1.1026 - accuracy: 0.8979 - val_loss: 0.3237 - val_accuracy: 0.9010
Epoch 832/1000
235/235 [==============================] - 0s 120us/step - loss: 1.7218 - accuracy: 0.8426 - val_loss: 0.3224 - val_accuracy: 0.9010
Epoch 833/1000
235/235 [==============================] - 0s 117us/step - loss: 1.7539 - accuracy: 0.8553 - val_loss: 0.3225 - 

235/235 [==============================] - 0s 116us/step - loss: 1.9818 - accuracy: 0.8255 - val_loss: 0.3275 - val_accuracy: 0.8911
Epoch 883/1000
235/235 [==============================] - 0s 117us/step - loss: 1.4622 - accuracy: 0.8596 - val_loss: 0.3249 - val_accuracy: 0.9010
Epoch 884/1000
235/235 [==============================] - 0s 128us/step - loss: 1.8756 - accuracy: 0.8511 - val_loss: 0.3244 - val_accuracy: 0.9010
Epoch 885/1000
235/235 [==============================] - 0s 110us/step - loss: 1.3617 - accuracy: 0.8809 - val_loss: 0.3211 - val_accuracy: 0.9010
Epoch 886/1000
235/235 [==============================] - 0s 125us/step - loss: 1.6698 - accuracy: 0.8681 - val_loss: 0.3215 - val_accuracy: 0.9010
Epoch 887/1000
235/235 [==============================] - 0s 118us/step - loss: 1.8345 - accuracy: 0.8426 - val_loss: 0.3221 - val_accuracy: 0.9010
Epoch 888/1000
235/235 [==============================] - 0s 118us/step - loss: 1.2245 - accuracy: 0.8936 - val_loss: 0.3220 - 

235/235 [==============================] - 0s 106us/step - loss: 1.9474 - accuracy: 0.8426 - val_loss: 0.3240 - val_accuracy: 0.9010
Epoch 938/1000
235/235 [==============================] - 0s 137us/step - loss: 1.4352 - accuracy: 0.8723 - val_loss: 0.3220 - val_accuracy: 0.9010
Epoch 939/1000
235/235 [==============================] - 0s 108us/step - loss: 1.5151 - accuracy: 0.8596 - val_loss: 0.3139 - val_accuracy: 0.9109
Epoch 940/1000
235/235 [==============================] - 0s 114us/step - loss: 2.0835 - accuracy: 0.8298 - val_loss: 0.3139 - val_accuracy: 0.9109
Epoch 941/1000
235/235 [==============================] - 0s 117us/step - loss: 1.3086 - accuracy: 0.8766 - val_loss: 0.3124 - val_accuracy: 0.9208
Epoch 942/1000
235/235 [==============================] - 0s 109us/step - loss: 1.4306 - accuracy: 0.8723 - val_loss: 0.3141 - val_accuracy: 0.9109
Epoch 943/1000
235/235 [==============================] - 0s 110us/step - loss: 2.3442 - accuracy: 0.8128 - val_loss: 0.3145 - 

235/235 [==============================] - 0s 106us/step - loss: 1.8256 - accuracy: 0.8468 - val_loss: 0.3155 - val_accuracy: 0.9208
Epoch 993/1000
235/235 [==============================] - 0s 120us/step - loss: 1.4354 - accuracy: 0.8723 - val_loss: 0.3169 - val_accuracy: 0.9109
Epoch 994/1000
235/235 [==============================] - 0s 120us/step - loss: 1.4432 - accuracy: 0.8681 - val_loss: 0.3187 - val_accuracy: 0.9109
Epoch 995/1000
235/235 [==============================] - 0s 117us/step - loss: 2.1699 - accuracy: 0.8426 - val_loss: 0.3169 - val_accuracy: 0.9109
Epoch 996/1000
235/235 [==============================] - 0s 112us/step - loss: 1.8249 - accuracy: 0.8468 - val_loss: 0.3181 - val_accuracy: 0.9109
Epoch 997/1000
235/235 [==============================] - 0s 113us/step - loss: 1.3043 - accuracy: 0.8809 - val_loss: 0.3151 - val_accuracy: 0.9208
Epoch 998/1000
235/235 [==============================] - 0s 118us/step - loss: 1.5166 - accuracy: 0.8596 - val_loss: 0.3164 - 

In [26]:
acc_test_comb2 = model1_comb2.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))

101/101 [==============================] - 0s 119us/step
combination test accuracy: 91.09%


In [27]:
#### improve neural network
model1_comb3 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [28]:
model1_comb3.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [29]:
model1_comb3.fit(X_train_comb, y_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_test_comb, y_test_comb))

Train on 235 samples, validate on 101 samples
Epoch 1/1000
235/235 [==============================] - 0s 912us/step - loss: 3.9170 - accuracy: 0.4511 - val_loss: 0.7456 - val_accuracy: 0.4950
Epoch 2/1000
235/235 [==============================] - 0s 143us/step - loss: 2.0675 - accuracy: 0.5021 - val_loss: 0.7077 - val_accuracy: 0.6634
Epoch 3/1000
235/235 [==============================] - 0s 166us/step - loss: 1.8729 - accuracy: 0.4766 - val_loss: 0.6958 - val_accuracy: 0.7129
Epoch 4/1000
235/235 [==============================] - 0s 171us/step - loss: 1.7692 - accuracy: 0.5106 - val_loss: 0.6912 - val_accuracy: 0.7228
Epoch 5/1000
235/235 [==============================] - 0s 150us/step - loss: 1.8875 - accuracy: 0.5021 - val_loss: 0.6874 - val_accuracy: 0.6535
Epoch 6/1000
235/235 [==============================] - 0s 147us/step - loss: 2.2871 - accuracy: 0.4511 - val_loss: 0.6853 - val_accuracy: 0.6238
Epoch 7/1000
235/235 [==============================] - 0s 162us/step - loss: 

235/235 [==============================] - 0s 158us/step - loss: 1.3210 - accuracy: 0.7830 - val_loss: 0.4983 - val_accuracy: 0.7228
Epoch 57/1000
235/235 [==============================] - 0s 163us/step - loss: 1.6427 - accuracy: 0.7532 - val_loss: 0.4915 - val_accuracy: 0.7228
Epoch 58/1000
235/235 [==============================] - 0s 126us/step - loss: 1.6111 - accuracy: 0.7915 - val_loss: 0.4897 - val_accuracy: 0.7228
Epoch 59/1000
235/235 [==============================] - 0s 118us/step - loss: 1.4973 - accuracy: 0.8000 - val_loss: 0.4939 - val_accuracy: 0.7228
Epoch 60/1000
235/235 [==============================] - 0s 137us/step - loss: 1.7477 - accuracy: 0.8511 - val_loss: 0.4909 - val_accuracy: 0.7228
Epoch 61/1000
235/235 [==============================] - 0s 147us/step - loss: 1.6225 - accuracy: 0.8511 - val_loss: 0.4874 - val_accuracy: 0.7228
Epoch 62/1000
235/235 [==============================] - 0s 135us/step - loss: 1.4827 - accuracy: 0.8723 - val_loss: 0.4936 - val_ac

Epoch 112/1000
235/235 [==============================] - 0s 139us/step - loss: 1.6602 - accuracy: 0.8596 - val_loss: 0.3616 - val_accuracy: 0.7525
Epoch 113/1000
235/235 [==============================] - 0s 154us/step - loss: 1.5927 - accuracy: 0.8596 - val_loss: 0.3610 - val_accuracy: 0.7525
Epoch 114/1000
235/235 [==============================] - 0s 148us/step - loss: 1.7240 - accuracy: 0.8511 - val_loss: 0.3545 - val_accuracy: 0.9208
Epoch 115/1000
235/235 [==============================] - 0s 138us/step - loss: 1.8656 - accuracy: 0.8383 - val_loss: 0.3553 - val_accuracy: 0.9208
Epoch 116/1000
235/235 [==============================] - 0s 129us/step - loss: 1.4036 - accuracy: 0.8766 - val_loss: 0.3576 - val_accuracy: 0.9208
Epoch 117/1000
235/235 [==============================] - 0s 114us/step - loss: 1.7219 - accuracy: 0.8511 - val_loss: 0.3957 - val_accuracy: 0.9208
Epoch 118/1000
235/235 [==============================] - 0s 123us/step - loss: 1.6709 - accuracy: 0.8468 - val_

235/235 [==============================] - 0s 105us/step - loss: 1.6860 - accuracy: 0.8511 - val_loss: 0.2995 - val_accuracy: 0.9208
Epoch 168/1000
235/235 [==============================] - 0s 139us/step - loss: 1.6647 - accuracy: 0.8681 - val_loss: 0.2966 - val_accuracy: 0.9208
Epoch 169/1000
235/235 [==============================] - 0s 120us/step - loss: 1.6165 - accuracy: 0.8596 - val_loss: 0.3084 - val_accuracy: 0.9208
Epoch 170/1000
235/235 [==============================] - 0s 114us/step - loss: 1.8120 - accuracy: 0.8468 - val_loss: 0.3013 - val_accuracy: 0.9208
Epoch 171/1000
235/235 [==============================] - 0s 115us/step - loss: 1.4175 - accuracy: 0.8723 - val_loss: 0.3031 - val_accuracy: 0.9208
Epoch 172/1000
235/235 [==============================] - 0s 116us/step - loss: 1.6178 - accuracy: 0.8553 - val_loss: 0.2995 - val_accuracy: 0.9208
Epoch 173/1000
235/235 [==============================] - 0s 118us/step - loss: 1.6160 - accuracy: 0.8596 - val_loss: 0.3678 - 

235/235 [==============================] - 0s 126us/step - loss: 1.9454 - accuracy: 0.8255 - val_loss: 0.2962 - val_accuracy: 0.9208
Epoch 223/1000
235/235 [==============================] - 0s 131us/step - loss: 1.7303 - accuracy: 0.8468 - val_loss: 0.2873 - val_accuracy: 0.9208
Epoch 224/1000
235/235 [==============================] - 0s 154us/step - loss: 1.2671 - accuracy: 0.8851 - val_loss: 0.2953 - val_accuracy: 0.9208
Epoch 225/1000
235/235 [==============================] - 0s 130us/step - loss: 1.5507 - accuracy: 0.8553 - val_loss: 0.2957 - val_accuracy: 0.9208
Epoch 226/1000
235/235 [==============================] - 0s 117us/step - loss: 1.4560 - accuracy: 0.8809 - val_loss: 0.3021 - val_accuracy: 0.9208
Epoch 227/1000
235/235 [==============================] - 0s 137us/step - loss: 1.4123 - accuracy: 0.8638 - val_loss: 0.2926 - val_accuracy: 0.9208
Epoch 228/1000
235/235 [==============================] - 0s 114us/step - loss: 1.4838 - accuracy: 0.8596 - val_loss: 0.3001 - 

...

235/235 [==============================] - 0s 109us/step - loss: 1.4822 - accuracy: 0.8681 - val_loss: 0.2928 - val_accuracy: 0.9208
Epoch 773/1000
235/235 [==============================] - 0s 112us/step - loss: 1.6209 - accuracy: 0.8553 - val_loss: 0.2917 - val_accuracy: 0.9208
Epoch 774/1000
235/235 [==============================] - 0s 112us/step - loss: 1.4918 - accuracy: 0.8638 - val_loss: 0.2935 - val_accuracy: 0.9208
Epoch 775/1000
235/235 [==============================] - 0s 116us/step - loss: 2.0125 - accuracy: 0.8298 - val_loss: 0.2959 - val_accuracy: 0.9208
Epoch 776/1000
235/235 [==============================] - 0s 109us/step - loss: 1.4270 - accuracy: 0.8681 - val_loss: 0.2938 - val_accuracy: 0.9208
Epoch 777/1000
235/235 [==============================] - 0s 113us/step - loss: 1.4252 - accuracy: 0.8681 - val_loss: 0.2920 - val_accuracy: 0.9208
Epoch 778/1000
235/235 [==============================] - 0s 115us/step - loss: 2.1340 - accuracy: 0.8255 - val_loss: 0.2933 - 

235/235 [==============================] - 0s 114us/step - loss: 1.9856 - accuracy: 0.8426 - val_loss: 0.2923 - val_accuracy: 0.9208
Epoch 828/1000
235/235 [==============================] - 0s 116us/step - loss: 1.4153 - accuracy: 0.8723 - val_loss: 0.2915 - val_accuracy: 0.9208
Epoch 829/1000
235/235 [==============================] - 0s 112us/step - loss: 1.5658 - accuracy: 0.8809 - val_loss: 0.2914 - val_accuracy: 0.9208
Epoch 830/1000
235/235 [==============================] - 0s 117us/step - loss: 1.6114 - accuracy: 0.8596 - val_loss: 0.2898 - val_accuracy: 0.9208
Epoch 831/1000
235/235 [==============================] - 0s 112us/step - loss: 1.6489 - accuracy: 0.8681 - val_loss: 0.2892 - val_accuracy: 0.9208
Epoch 832/1000
235/235 [==============================] - 0s 194us/step - loss: 1.4804 - accuracy: 0.8681 - val_loss: 0.2900 - val_accuracy: 0.9208
Epoch 833/1000
235/235 [==============================] - 0s 250us/step - loss: 1.9379 - accuracy: 0.8383 - val_loss: 0.2919 - 

235/235 [==============================] - 0s 113us/step - loss: 1.8069 - accuracy: 0.8468 - val_loss: 0.2894 - val_accuracy: 0.9208
Epoch 883/1000
235/235 [==============================] - 0s 115us/step - loss: 1.2856 - accuracy: 0.8809 - val_loss: 0.2894 - val_accuracy: 0.9208
Epoch 884/1000
235/235 [==============================] - 0s 130us/step - loss: 1.6116 - accuracy: 0.8596 - val_loss: 0.2891 - val_accuracy: 0.9208
Epoch 885/1000
235/235 [==============================] - 0s 121us/step - loss: 1.5544 - accuracy: 0.8596 - val_loss: 0.2898 - val_accuracy: 0.9208
Epoch 886/1000
235/235 [==============================] - 0s 120us/step - loss: 1.5364 - accuracy: 0.8681 - val_loss: 0.2896 - val_accuracy: 0.9208
Epoch 887/1000
235/235 [==============================] - 0s 116us/step - loss: 1.8816 - accuracy: 0.8383 - val_loss: 0.2906 - val_accuracy: 0.9208
Epoch 888/1000
235/235 [==============================] - 0s 108us/step - loss: 1.5353 - accuracy: 0.8681 - val_loss: 0.2901 - 

235/235 [==============================] - 0s 119us/step - loss: 2.0092 - accuracy: 0.8298 - val_loss: 0.2885 - val_accuracy: 0.9208
Epoch 938/1000
235/235 [==============================] - 0s 125us/step - loss: 2.0105 - accuracy: 0.8298 - val_loss: 0.2915 - val_accuracy: 0.9208
Epoch 939/1000
235/235 [==============================] - 0s 123us/step - loss: 1.6749 - accuracy: 0.8553 - val_loss: 0.2919 - val_accuracy: 0.9208
Epoch 940/1000
235/235 [==============================] - 0s 118us/step - loss: 1.6103 - accuracy: 0.8596 - val_loss: 0.2901 - val_accuracy: 0.9208
Epoch 941/1000
235/235 [==============================] - 0s 111us/step - loss: 1.4440 - accuracy: 0.8851 - val_loss: 0.2869 - val_accuracy: 0.9208
Epoch 942/1000
235/235 [==============================] - 0s 115us/step - loss: 1.9551 - accuracy: 0.8298 - val_loss: 0.2893 - val_accuracy: 0.9208
Epoch 943/1000
235/235 [==============================] - 0s 120us/step - loss: 2.0851 - accuracy: 0.8213 - val_loss: 0.2908 - 

235/235 [==============================] - 0s 114us/step - loss: 2.0492 - accuracy: 0.8383 - val_loss: 0.2924 - val_accuracy: 0.9208
Epoch 993/1000
235/235 [==============================] - 0s 125us/step - loss: 2.0079 - accuracy: 0.8298 - val_loss: 0.2924 - val_accuracy: 0.9208
Epoch 994/1000
235/235 [==============================] - 0s 127us/step - loss: 1.4149 - accuracy: 0.8723 - val_loss: 0.2927 - val_accuracy: 0.9208
Epoch 995/1000
235/235 [==============================] - 0s 124us/step - loss: 1.8055 - accuracy: 0.8468 - val_loss: 0.2944 - val_accuracy: 0.9208
Epoch 996/1000
235/235 [==============================] - 0s 114us/step - loss: 1.3393 - accuracy: 0.8809 - val_loss: 0.2933 - val_accuracy: 0.9208
Epoch 997/1000
235/235 [==============================] - 0s 108us/step - loss: 1.7878 - accuracy: 0.8553 - val_loss: 0.2907 - val_accuracy: 0.9208
Epoch 998/1000
235/235 [==============================] - 0s 116us/step - loss: 1.8144 - accuracy: 0.8426 - val_loss: 0.2903 - 

In [30]:
acc_test_comb3 = model1_comb3.evaluate(X_test_comb, y_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

101/101 [==============================] - 0s 103us/step
combination test accuracy: 92.08%


In [31]:
#### neural network on over-sampling data
model1_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [32]:
model1_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [33]:
model1_over.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 374us/step - loss: 4.9455 - accuracy: 0.5213 - val_loss: 0.6659 - val_accuracy: 0.5496
Epoch 2/1000
305/305 [==============================] - 0s 114us/step - loss: 0.6447 - accuracy: 0.5475 - val_loss: 0.6099 - val_accuracy: 0.7023
Epoch 3/1000
305/305 [==============================] - 0s 124us/step - loss: 0.6080 - accuracy: 0.5934 - val_loss: 0.5979 - val_accuracy: 0.7023
Epoch 4/1000
305/305 [==============================] - 0s 140us/step - loss: 0.5821 - accuracy: 0.6820 - val_loss: 0.5487 - val_accuracy: 0.7863
Epoch 5/1000
305/305 [==============================] - 0s 133us/step - loss: 0.5677 - accuracy: 0.6918 - val_loss: 0.5296 - val_accuracy: 0.7481
Epoch 6/1000
305/305 [==============================] - 0s 133us/step - loss: 0.5543 - accuracy: 0.7082 - val_loss: 0.5283 - val_accuracy: 0.7634
Epoch 7/1000
305/305 [==============================] - 0s 148us/step - loss: 

305/305 [==============================] - 0s 134us/step - loss: 0.3762 - accuracy: 0.8328 - val_loss: 0.3955 - val_accuracy: 0.8321
Epoch 57/1000
305/305 [==============================] - 0s 120us/step - loss: 0.3772 - accuracy: 0.8328 - val_loss: 0.3969 - val_accuracy: 0.8092
Epoch 58/1000
305/305 [==============================] - 0s 112us/step - loss: 0.3797 - accuracy: 0.8361 - val_loss: 0.3921 - val_accuracy: 0.8092
Epoch 59/1000
305/305 [==============================] - 0s 108us/step - loss: 0.3723 - accuracy: 0.8361 - val_loss: 0.4265 - val_accuracy: 0.7863
Epoch 60/1000
305/305 [==============================] - 0s 103us/step - loss: 0.3718 - accuracy: 0.8328 - val_loss: 0.3905 - val_accuracy: 0.8168
Epoch 61/1000
305/305 [==============================] - 0s 103us/step - loss: 0.3730 - accuracy: 0.8361 - val_loss: 0.3953 - val_accuracy: 0.8092
Epoch 62/1000
305/305 [==============================] - 0s 102us/step - loss: 0.3755 - accuracy: 0.8361 - val_loss: 0.3848 - val_ac

Epoch 112/1000
305/305 [==============================] - 0s 105us/step - loss: 0.3333 - accuracy: 0.8459 - val_loss: 0.3494 - val_accuracy: 0.8321
Epoch 113/1000
305/305 [==============================] - 0s 108us/step - loss: 0.3356 - accuracy: 0.8459 - val_loss: 0.3429 - val_accuracy: 0.8321
Epoch 114/1000
305/305 [==============================] - 0s 106us/step - loss: 0.3343 - accuracy: 0.8459 - val_loss: 0.3440 - val_accuracy: 0.8244
Epoch 115/1000
305/305 [==============================] - 0s 99us/step - loss: 0.3310 - accuracy: 0.8426 - val_loss: 0.3520 - val_accuracy: 0.8092
Epoch 116/1000
305/305 [==============================] - 0s 102us/step - loss: 0.3338 - accuracy: 0.8459 - val_loss: 0.3555 - val_accuracy: 0.8092
Epoch 117/1000
305/305 [==============================] - 0s 103us/step - loss: 0.3328 - accuracy: 0.8393 - val_loss: 0.3433 - val_accuracy: 0.8321
Epoch 118/1000
305/305 [==============================] - 0s 105us/step - loss: 0.3342 - accuracy: 0.8426 - val_l

305/305 [==============================] - 0s 103us/step - loss: 0.3085 - accuracy: 0.8492 - val_loss: 0.3625 - val_accuracy: 0.8015
Epoch 168/1000
305/305 [==============================] - 0s 111us/step - loss: 0.3100 - accuracy: 0.8492 - val_loss: 0.3428 - val_accuracy: 0.8092
Epoch 169/1000
305/305 [==============================] - 0s 106us/step - loss: 0.3065 - accuracy: 0.8492 - val_loss: 0.3402 - val_accuracy: 0.8092
Epoch 170/1000
305/305 [==============================] - 0s 103us/step - loss: 0.3098 - accuracy: 0.8492 - val_loss: 0.3319 - val_accuracy: 0.8092
Epoch 171/1000
305/305 [==============================] - 0s 98us/step - loss: 0.3120 - accuracy: 0.8492 - val_loss: 0.3421 - val_accuracy: 0.8092
Epoch 172/1000
305/305 [==============================] - 0s 98us/step - loss: 0.3056 - accuracy: 0.8492 - val_loss: 0.3191 - val_accuracy: 0.8321
Epoch 173/1000
305/305 [==============================] - 0s 101us/step - loss: 0.3069 - accuracy: 0.8492 - val_loss: 0.3207 - va

305/305 [==============================] - 0s 95us/step - loss: 0.2955 - accuracy: 0.8525 - val_loss: 0.3085 - val_accuracy: 0.8321
Epoch 223/1000
305/305 [==============================] - 0s 103us/step - loss: 0.2923 - accuracy: 0.8525 - val_loss: 0.3181 - val_accuracy: 0.8092
Epoch 224/1000
305/305 [==============================] - 0s 100us/step - loss: 0.2902 - accuracy: 0.8525 - val_loss: 0.3070 - val_accuracy: 0.8321
Epoch 225/1000
305/305 [==============================] - 0s 98us/step - loss: 0.2915 - accuracy: 0.8557 - val_loss: 0.3162 - val_accuracy: 0.8168
Epoch 226/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2898 - accuracy: 0.8492 - val_loss: 0.3164 - val_accuracy: 0.8244
Epoch 227/1000
305/305 [==============================] - 0s 90us/step - loss: 0.2944 - accuracy: 0.8557 - val_loss: 0.3530 - val_accuracy: 0.8092
Epoch 228/1000
305/305 [==============================] - 0s 87us/step - loss: 0.2963 - accuracy: 0.8525 - val_loss: 0.3055 - val_a

...

Epoch 724/1000
305/305 [==============================] - 0s 90us/step - loss: 0.2320 - accuracy: 0.8820 - val_loss: 0.2568 - val_accuracy: 0.8626
Epoch 725/1000
305/305 [==============================] - 0s 100us/step - loss: 0.2363 - accuracy: 0.8787 - val_loss: 0.2603 - val_accuracy: 0.8626
Epoch 726/1000
305/305 [==============================] - 0s 98us/step - loss: 0.2316 - accuracy: 0.8787 - val_loss: 0.2805 - val_accuracy: 0.8397
Epoch 727/1000
305/305 [==============================] - 0s 93us/step - loss: 0.2329 - accuracy: 0.8787 - val_loss: 0.2613 - val_accuracy: 0.8397
Epoch 728/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2361 - accuracy: 0.8787 - val_loss: 0.2688 - val_accuracy: 0.8397
Epoch 729/1000
305/305 [==============================] - 0s 90us/step - loss: 0.2341 - accuracy: 0.8754 - val_loss: 0.2558 - val_accuracy: 0.8626
Epoch 730/1000
305/305 [==============================] - 0s 94us/step - loss: 0.2307 - accuracy: 0.8787 - val_loss: 

Epoch 780/1000
305/305 [==============================] - 0s 101us/step - loss: 0.2341 - accuracy: 0.8754 - val_loss: 0.2575 - val_accuracy: 0.8626
Epoch 781/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2352 - accuracy: 0.8754 - val_loss: 0.2578 - val_accuracy: 0.8626
Epoch 782/1000
305/305 [==============================] - 0s 86us/step - loss: 0.2316 - accuracy: 0.8787 - val_loss: 0.2778 - val_accuracy: 0.8397
Epoch 783/1000
305/305 [==============================] - 0s 95us/step - loss: 0.2288 - accuracy: 0.8787 - val_loss: 0.2663 - val_accuracy: 0.8473
Epoch 784/1000
305/305 [==============================] - 0s 94us/step - loss: 0.2289 - accuracy: 0.8820 - val_loss: 0.2590 - val_accuracy: 0.8626
Epoch 785/1000
305/305 [==============================] - 0s 96us/step - loss: 0.2336 - accuracy: 0.8754 - val_loss: 0.2581 - val_accuracy: 0.8626
Epoch 786/1000
305/305 [==============================] - 0s 96us/step - loss: 0.2301 - accuracy: 0.8787 - val_loss:

Epoch 836/1000
305/305 [==============================] - 0s 92us/step - loss: 0.2328 - accuracy: 0.8787 - val_loss: 0.2634 - val_accuracy: 0.8550
Epoch 837/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2286 - accuracy: 0.8754 - val_loss: 0.2550 - val_accuracy: 0.8626
Epoch 838/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2331 - accuracy: 0.8787 - val_loss: 0.2633 - val_accuracy: 0.8473
Epoch 839/1000
305/305 [==============================] - 0s 95us/step - loss: 0.2293 - accuracy: 0.8787 - val_loss: 0.2695 - val_accuracy: 0.8397
Epoch 840/1000
305/305 [==============================] - 0s 94us/step - loss: 0.2303 - accuracy: 0.8787 - val_loss: 0.2802 - val_accuracy: 0.8397
Epoch 841/1000
305/305 [==============================] - 0s 91us/step - loss: 0.2307 - accuracy: 0.8754 - val_loss: 0.3213 - val_accuracy: 0.8321
Epoch 842/1000
305/305 [==============================] - 0s 88us/step - loss: 0.2334 - accuracy: 0.8754 - val_loss: 

Epoch 892/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2299 - accuracy: 0.8787 - val_loss: 0.2710 - val_accuracy: 0.8473
Epoch 893/1000
305/305 [==============================] - 0s 91us/step - loss: 0.2306 - accuracy: 0.8787 - val_loss: 0.2744 - val_accuracy: 0.8473
Epoch 894/1000
305/305 [==============================] - 0s 96us/step - loss: 0.2295 - accuracy: 0.8754 - val_loss: 0.2584 - val_accuracy: 0.8626
Epoch 895/1000
305/305 [==============================] - 0s 98us/step - loss: 0.2289 - accuracy: 0.8754 - val_loss: 0.2674 - val_accuracy: 0.8473
Epoch 896/1000
305/305 [==============================] - 0s 98us/step - loss: 0.2275 - accuracy: 0.8820 - val_loss: 0.2559 - val_accuracy: 0.8626
Epoch 897/1000
305/305 [==============================] - 0s 95us/step - loss: 0.2319 - accuracy: 0.8754 - val_loss: 0.2604 - val_accuracy: 0.8550
Epoch 898/1000
305/305 [==============================] - 0s 93us/step - loss: 0.2286 - accuracy: 0.8754 - val_loss: 0

Epoch 948/1000
305/305 [==============================] - 0s 97us/step - loss: 0.2301 - accuracy: 0.8787 - val_loss: 0.2551 - val_accuracy: 0.8626
Epoch 949/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2292 - accuracy: 0.8787 - val_loss: 0.2638 - val_accuracy: 0.8473
Epoch 950/1000
305/305 [==============================] - 0s 101us/step - loss: 0.2285 - accuracy: 0.8787 - val_loss: 0.2824 - val_accuracy: 0.8397
Epoch 951/1000
305/305 [==============================] - 0s 96us/step - loss: 0.2313 - accuracy: 0.8754 - val_loss: 0.2865 - val_accuracy: 0.8397
Epoch 952/1000
305/305 [==============================] - 0s 95us/step - loss: 0.2249 - accuracy: 0.8787 - val_loss: 0.2802 - val_accuracy: 0.8473
Epoch 953/1000
305/305 [==============================] - 0s 88us/step - loss: 0.2337 - accuracy: 0.8754 - val_loss: 0.2568 - val_accuracy: 0.8550
Epoch 954/1000
305/305 [==============================] - 0s 94us/step - loss: 0.2304 - accuracy: 0.8787 - val_loss:

In [34]:
acc_test_over = model1_over.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over*100))

131/131 [==============================] - 0s 82us/step
over-sampling test accuracy: 86.26%


In [35]:
#### improve neural network on over-sampling data
model1_over2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_train_over.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
])

In [36]:
model1_over2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [37]:
model1_over2.fit(X_train_over, y_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_test_over, y_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 476us/step - loss: 13.9397 - accuracy: 0.4590 - val_loss: 0.6992 - val_accuracy: 0.4962
Epoch 2/1000
305/305 [==============================] - 0s 104us/step - loss: 0.7022 - accuracy: 0.4918 - val_loss: 0.6935 - val_accuracy: 0.6107
Epoch 3/1000
305/305 [==============================] - 0s 116us/step - loss: 0.6903 - accuracy: 0.5770 - val_loss: 0.6756 - val_accuracy: 0.6641
Epoch 4/1000
305/305 [==============================] - 0s 108us/step - loss: 0.6846 - accuracy: 0.6525 - val_loss: 0.6777 - val_accuracy: 0.6794
Epoch 5/1000
305/305 [==============================] - 0s 126us/step - loss: 0.6783 - accuracy: 0.6557 - val_loss: 0.6661 - val_accuracy: 0.6718
Epoch 6/1000
305/305 [==============================] - 0s 124us/step - loss: 0.6719 - accuracy: 0.7115 - val_loss: 0.6778 - val_accuracy: 0.5725
Epoch 7/1000
305/305 [==============================] - 0s 111us/step - loss:

305/305 [==============================] - 0s 122us/step - loss: 0.4167 - accuracy: 0.8197 - val_loss: 0.4186 - val_accuracy: 0.8015
Epoch 57/1000
305/305 [==============================] - 0s 129us/step - loss: 0.4049 - accuracy: 0.8230 - val_loss: 0.4210 - val_accuracy: 0.7939
Epoch 58/1000
305/305 [==============================] - 0s 106us/step - loss: 0.4041 - accuracy: 0.8230 - val_loss: 0.4196 - val_accuracy: 0.8015
Epoch 59/1000
305/305 [==============================] - 0s 118us/step - loss: 0.4000 - accuracy: 0.8197 - val_loss: 0.4185 - val_accuracy: 0.8015
Epoch 60/1000
305/305 [==============================] - 0s 109us/step - loss: 0.4001 - accuracy: 0.8295 - val_loss: 0.4172 - val_accuracy: 0.8015
Epoch 61/1000
305/305 [==============================] - 0s 105us/step - loss: 0.3967 - accuracy: 0.8262 - val_loss: 0.4293 - val_accuracy: 0.7939
Epoch 62/1000
305/305 [==============================] - 0s 106us/step - loss: 0.4000 - accuracy: 0.8230 - val_loss: 0.4133 - val_ac

305/305 [==============================] - 0s 95us/step - loss: 0.3425 - accuracy: 0.8426 - val_loss: 0.4072 - val_accuracy: 0.8092
Epoch 112/1000
305/305 [==============================] - 0s 106us/step - loss: 0.3436 - accuracy: 0.8426 - val_loss: 0.3874 - val_accuracy: 0.8015
Epoch 113/1000
305/305 [==============================] - 0s 118us/step - loss: 0.3474 - accuracy: 0.8393 - val_loss: 0.3590 - val_accuracy: 0.8168
Epoch 114/1000
305/305 [==============================] - 0s 99us/step - loss: 0.3398 - accuracy: 0.8525 - val_loss: 0.3977 - val_accuracy: 0.8168
Epoch 115/1000
305/305 [==============================] - 0s 83us/step - loss: 0.3360 - accuracy: 0.8393 - val_loss: 0.4518 - val_accuracy: 0.7863
Epoch 116/1000
305/305 [==============================] - 0s 92us/step - loss: 0.3403 - accuracy: 0.8361 - val_loss: 0.4020 - val_accuracy: 0.8092
Epoch 117/1000
305/305 [==============================] - 0s 102us/step - loss: 0.3397 - accuracy: 0.8426 - val_loss: 0.3736 - val_

305/305 [==============================] - 0s 94us/step - loss: 0.3053 - accuracy: 0.8492 - val_loss: 0.3795 - val_accuracy: 0.8015
Epoch 167/1000
305/305 [==============================] - 0s 104us/step - loss: 0.3241 - accuracy: 0.8459 - val_loss: 0.3440 - val_accuracy: 0.8015
Epoch 168/1000
305/305 [==============================] - 0s 112us/step - loss: 0.3214 - accuracy: 0.8393 - val_loss: 0.3984 - val_accuracy: 0.8092
Epoch 169/1000
305/305 [==============================] - 0s 99us/step - loss: 0.3048 - accuracy: 0.8590 - val_loss: 0.3465 - val_accuracy: 0.8168
Epoch 170/1000
305/305 [==============================] - 0s 104us/step - loss: 0.3041 - accuracy: 0.8492 - val_loss: 0.3961 - val_accuracy: 0.8092
Epoch 171/1000
305/305 [==============================] - 0s 102us/step - loss: 0.3029 - accuracy: 0.8492 - val_loss: 0.3474 - val_accuracy: 0.8168
Epoch 172/1000
305/305 [==============================] - 0s 104us/step - loss: 0.3084 - accuracy: 0.8492 - val_loss: 0.3317 - va

305/305 [==============================] - 0s 91us/step - loss: 0.2943 - accuracy: 0.8525 - val_loss: 0.3683 - val_accuracy: 0.8168
Epoch 222/1000
305/305 [==============================] - 0s 94us/step - loss: 0.2832 - accuracy: 0.8525 - val_loss: 0.3131 - val_accuracy: 0.8321
Epoch 223/1000
305/305 [==============================] - 0s 106us/step - loss: 0.2835 - accuracy: 0.8590 - val_loss: 0.3152 - val_accuracy: 0.8244
Epoch 224/1000
305/305 [==============================] - 0s 95us/step - loss: 0.2880 - accuracy: 0.8557 - val_loss: 0.3176 - val_accuracy: 0.8321
Epoch 225/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2823 - accuracy: 0.8623 - val_loss: 0.3526 - val_accuracy: 0.8092
Epoch 226/1000
305/305 [==============================] - 0s 100us/step - loss: 0.2760 - accuracy: 0.8590 - val_loss: 0.3852 - val_accuracy: 0.8015
Epoch 227/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2960 - accuracy: 0.8557 - val_loss: 0.3437 - val_a

...

Epoch 777/1000
305/305 [==============================] - 0s 103us/step - loss: 0.2191 - accuracy: 0.8852 - val_loss: 0.3100 - val_accuracy: 0.8702
Epoch 778/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2245 - accuracy: 0.8918 - val_loss: 0.3078 - val_accuracy: 0.8779
Epoch 779/1000
305/305 [==============================] - 0s 94us/step - loss: 0.2150 - accuracy: 0.8984 - val_loss: 0.2777 - val_accuracy: 0.8550
Epoch 780/1000
305/305 [==============================] - 0s 92us/step - loss: 0.2248 - accuracy: 0.8852 - val_loss: 0.2770 - val_accuracy: 0.8779
Epoch 781/1000
305/305 [==============================] - 0s 94us/step - loss: 0.2267 - accuracy: 0.8820 - val_loss: 0.2764 - val_accuracy: 0.8702
Epoch 782/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2162 - accuracy: 0.8918 - val_loss: 0.3106 - val_accuracy: 0.8702
Epoch 783/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2200 - accuracy: 0.8918 - val_los

305/305 [==============================] - 0s 101us/step - loss: 0.2228 - accuracy: 0.8787 - val_loss: 0.2789 - val_accuracy: 0.8702
Epoch 833/1000
305/305 [==============================] - 0s 122us/step - loss: 0.2320 - accuracy: 0.8820 - val_loss: 0.2745 - val_accuracy: 0.8702
Epoch 834/1000
305/305 [==============================] - 0s 128us/step - loss: 0.2368 - accuracy: 0.8787 - val_loss: 0.2732 - val_accuracy: 0.8702
Epoch 835/1000
305/305 [==============================] - ETA: 0s - loss: 0.2542 - accuracy: 0.81 - 0s 160us/step - loss: 0.2213 - accuracy: 0.8885 - val_loss: 0.4656 - val_accuracy: 0.8092
Epoch 836/1000
305/305 [==============================] - 0s 180us/step - loss: 0.2276 - accuracy: 0.8885 - val_loss: 0.2978 - val_accuracy: 0.8779
Epoch 837/1000
305/305 [==============================] - 0s 186us/step - loss: 0.2235 - accuracy: 0.8820 - val_loss: 0.2961 - val_accuracy: 0.8626
Epoch 838/1000
305/305 [==============================] - 0s 155us/step - loss: 0.218

305/305 [==============================] - 0s 89us/step - loss: 0.2352 - accuracy: 0.8754 - val_loss: 0.2769 - val_accuracy: 0.8702
Epoch 888/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2202 - accuracy: 0.8885 - val_loss: 0.2913 - val_accuracy: 0.8626
Epoch 889/1000
305/305 [==============================] - 0s 102us/step - loss: 0.2160 - accuracy: 0.8885 - val_loss: 0.2762 - val_accuracy: 0.8702
Epoch 890/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2163 - accuracy: 0.8918 - val_loss: 0.2695 - val_accuracy: 0.8702
Epoch 891/1000
305/305 [==============================] - 0s 100us/step - loss: 0.2426 - accuracy: 0.8787 - val_loss: 0.2693 - val_accuracy: 0.8702
Epoch 892/1000
305/305 [==============================] - 0s 98us/step - loss: 0.2259 - accuracy: 0.8885 - val_loss: 0.3253 - val_accuracy: 0.8473
Epoch 893/1000
305/305 [==============================] - 0s 98us/step - loss: 0.2325 - accuracy: 0.8787 - val_loss: 0.3614 - val_

Epoch 943/1000
305/305 [==============================] - 0s 99us/step - loss: 0.2165 - accuracy: 0.8885 - val_loss: 0.3231 - val_accuracy: 0.8473
Epoch 944/1000
305/305 [==============================] - 0s 104us/step - loss: 0.2196 - accuracy: 0.8918 - val_loss: 0.4168 - val_accuracy: 0.8321
Epoch 945/1000
305/305 [==============================] - 0s 100us/step - loss: 0.2303 - accuracy: 0.8820 - val_loss: 0.2940 - val_accuracy: 0.8626
Epoch 946/1000
305/305 [==============================] - 0s 96us/step - loss: 0.2181 - accuracy: 0.8852 - val_loss: 0.2909 - val_accuracy: 0.8702
Epoch 947/1000
305/305 [==============================] - 0s 96us/step - loss: 0.2265 - accuracy: 0.8885 - val_loss: 0.6457 - val_accuracy: 0.7557
Epoch 948/1000
305/305 [==============================] - 0s 96us/step - loss: 0.2335 - accuracy: 0.8820 - val_loss: 0.3262 - val_accuracy: 0.8702
Epoch 949/1000
305/305 [==============================] - 0s 89us/step - loss: 0.2192 - accuracy: 0.8885 - val_loss:

305/305 [==============================] - 0s 107us/step - loss: 0.2117 - accuracy: 0.8951 - val_loss: 0.2874 - val_accuracy: 0.8702
Epoch 999/1000
305/305 [==============================] - 0s 107us/step - loss: 0.2176 - accuracy: 0.8951 - val_loss: 0.3464 - val_accuracy: 0.8550
Epoch 1000/1000
305/305 [==============================] - 0s 107us/step - loss: 0.2247 - accuracy: 0.8852 - val_loss: 0.2862 - val_accuracy: 0.8702


In [39]:
acc_test_over2 = model1_over2.evaluate(X_test_over, y_test_over)[1]
print('over-sampling test accuracy: %.2f%%' % (acc_test_over2*100))

131/131 [==============================] - 0s 117us/step
over-sampling test accuracy: 87.02%


In [40]:
############ Feature selection using lasso ##########

In [41]:
## Retrieved from https://towardsdatascience.com/feature-selection-using-regularisation-a3678b71e499
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import StandardScaler
import numpy as np

In [42]:
selection = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
selection.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False,
                                             fit_intercept=True,
                                             intercept_scaling=1, l1_ratio=None,
                                             max_iter=100, multi_class='warn',
                                             n_jobs=None, penalty='l1',
                                             random_state=None, solver='warn',
                                             tol=0.0001, verbose=0,
                                             warm_start=False),
                max_features=None, norm_order=1, prefit=False, threshold=None)

In [43]:
names = np.array(df_clean.columns).tolist()
names.remove('pheno')

In [44]:
X_train_features_comb = np.vstack((names, X_train_comb))
X_train_features_comb = pd.DataFrame(X_train_features_comb)

In [45]:
sel_features = X_train_features_comb.columns[(selection.get_support())]
print('total features: {}'.format((X_train_features_comb.shape[1])))
print('selected features: {}'.format(len(sel_features)))

total features: 974
selected features: 37


In [46]:
cols = sel_features.values
cols.reshape((1, -1))

array([[ 26,  32,  42,  76,  83,  84,  85,  86, 135, 137, 162, 182, 191,
        224, 273, 305, 393, 461, 475, 480, 494, 543, 553, 620, 621, 634,
        650, 672, 756, 847, 851, 852, 853, 854, 857, 972, 973]])

In [47]:
names_arr = np.array(names)
names_arr[cols]

array(['TTTTAATAGCTAGCACTTAATTGTGTTGGCTATTTTTTATGTCCAAAACGTGCTGATGACATAAAAAGCACGCATGGAAAAACAGTCGACAGACTATAAA',
       'TTTGCCAGTATC', 'TTTCAGCGACT', 'TTGTAAATAAGC', 'TTGCAAATCCAC',
       'TTGATGAAGCAGTAGGG', 'TTGATGAAGCAGTAGGGA',
       'TTGATGAAGCAGTAGGGATTATG', 'TTATCATCAAATG',
       'TTATAGTCTGTCGACTGTTTTTCCATGCGTGCTTTTTATGTCATCAGCACGTTTTGGACATAAAAAATAGCCAACACAATTAAGTGCTAGCTATTAAAAG',
       'TTAATTGAATAACGGGAAGTAGCTCAGCTTGGTAGAGCACTTGGTTTGGGACCAAGGGGTCGCAGGTTCGAATCCTGTCTTCCCGATTACTTCTTAAATT',
       'TGTTAAAAACTA', 'TGTCTGATTTTT', 'TGCTACCGCA',
       'TCTTTTAATAGCTAGCACTTAATTGTGTTGGCTATTTTTTATGTCCAAAACGTGCTGATGACATAAAAAGCACGCATGGAAAAACAGTCGACAGACTATA',
       'TCGCTGAAATAT', 'TAGATTCAAATAT', 'GTTGCAAATCCAC', 'GTGATGTTGTTTGT',
       'GTGAATTCATG', 'GTAGCTGGATT', 'GCTTGACTGT', 'GCTAATTATGTTTCTGGATT',
       'CTTGATGAAGCAGTAGGG', 'CTTGATGAAGCAGTAGGGATTATG',
       'CTCTTTTAATAGCTAGCACTTAATTGTGTTGGCTATTTTTTATGTCCAAAACGTGCTGATGACATAAAAAGCACGCATGGAAAAACAGTCGACAGACTAT',
       'CTAATG

In [48]:
removed_features = X_train_features_comb.columns[(selection.estimator_.coef_ == 0).ravel().tolist()]
removed_features

Int64Index([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,
            ...
            962, 963, 964, 965, 966, 967, 968, 969, 970, 971],
           dtype='int64', length=937)

In [49]:
len(removed_features)

937

In [50]:
###### keep selected variables as a new dataframe
df_sel = df_clean.loc[:,names_arr[cols]].copy()
df_sel['pheno'] = df_clean['pheno']
X_sel = df_sel.loc[:, df_sel.columns != 'pheno'].values
y_sel = df_sel['pheno'].values
print(X_sel.shape, y_sel.shape, df_sel.shape)

(255, 37) (255,) (255, 38)


In [51]:
# combination of under- and over- sampling
from collections import Counter
from sklearn.datasets import make_classification
from imblearn.combine import SMOTEENN
smote_enn = SMOTEENN(random_state=100)
X_sel_comb, y_sel_comb = smote_enn.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_comb).items()))

[(0, 163), (1, 166)]


In [52]:
# over-sampling
from imblearn.over_sampling import RandomOverSampler
overS = RandomOverSampler(random_state=100)
X_sel_over, y_sel_over = overS.fit_resample(X_sel, y_sel)
print(sorted(Counter(y_sel_over).items()))

[(0, 218), (1, 218)]


In [53]:
# split into train, validation, and test data (combination)
from sklearn.model_selection import train_test_split
X_sel_train_comb, X_sel_test_comb, y_sel_train_comb, y_sel_test_comb = train_test_split(X_sel_comb, y_sel_comb,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_comb)

In [54]:
# split into train, test data (over)
from sklearn.model_selection import train_test_split
X_sel_train_over, X_sel_test_over, y_sel_train_over, y_sel_test_over = train_test_split(X_sel_over, y_sel_over,
                                                    test_size = 0.3,
                                                    random_state=123,
                                                    stratify=y_sel_over)

In [55]:
#### neural network on combination data
model2_comb = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [56]:
model2_comb.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [57]:
model2_comb.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 230 samples, validate on 99 samples
Epoch 1/1000
230/230 [==============================] - 0s 447us/step - loss: 13.2692 - accuracy: 0.4522 - val_loss: 0.7982 - val_accuracy: 0.3939
Epoch 2/1000
230/230 [==============================] - 0s 87us/step - loss: 0.7499 - accuracy: 0.3696 - val_loss: 0.7250 - val_accuracy: 0.4141
Epoch 3/1000
230/230 [==============================] - 0s 117us/step - loss: 0.7068 - accuracy: 0.3957 - val_loss: 0.6979 - val_accuracy: 0.4040
Epoch 4/1000
230/230 [==============================] - 0s 121us/step - loss: 0.6906 - accuracy: 0.4174 - val_loss: 0.6866 - val_accuracy: 0.3838
Epoch 5/1000
230/230 [==============================] - 0s 116us/step - loss: 0.6714 - accuracy: 0.3478 - val_loss: 0.6759 - val_accuracy: 0.3939
Epoch 6/1000
230/230 [==============================] - 0s 122us/step - loss: 0.6591 - accuracy: 0.4174 - val_loss: 0.6943 - val_accuracy: 0.4545
Epoch 7/1000
230/230 [==============================] - 0s 147us/step - loss: 0

230/230 [==============================] - 0s 173us/step - loss: 0.3073 - accuracy: 0.8304 - val_loss: 0.3521 - val_accuracy: 0.7980
Epoch 57/1000
230/230 [==============================] - 0s 150us/step - loss: 0.3026 - accuracy: 0.8348 - val_loss: 0.3492 - val_accuracy: 0.7980
Epoch 58/1000
230/230 [==============================] - 0s 142us/step - loss: 0.2986 - accuracy: 0.8522 - val_loss: 0.3457 - val_accuracy: 0.8586
Epoch 59/1000
230/230 [==============================] - 0s 108us/step - loss: 0.2968 - accuracy: 0.8565 - val_loss: 0.3431 - val_accuracy: 0.8081
Epoch 60/1000
230/230 [==============================] - 0s 99us/step - loss: 0.2911 - accuracy: 0.8391 - val_loss: 0.3399 - val_accuracy: 0.9091
Epoch 61/1000
230/230 [==============================] - 0s 91us/step - loss: 0.2875 - accuracy: 0.9391 - val_loss: 0.3372 - val_accuracy: 0.9091
Epoch 62/1000
230/230 [==============================] - 0s 100us/step - loss: 0.2844 - accuracy: 0.9435 - val_loss: 0.3346 - val_accu

230/230 [==============================] - 0s 90us/step - loss: 0.2094 - accuracy: 0.9435 - val_loss: 0.2842 - val_accuracy: 0.9091
Epoch 113/1000
230/230 [==============================] - 0s 96us/step - loss: 0.2077 - accuracy: 0.9435 - val_loss: 0.2970 - val_accuracy: 0.8889
Epoch 114/1000
230/230 [==============================] - 0s 96us/step - loss: 0.2105 - accuracy: 0.9435 - val_loss: 0.2833 - val_accuracy: 0.9091
Epoch 115/1000
230/230 [==============================] - 0s 92us/step - loss: 0.2066 - accuracy: 0.9435 - val_loss: 0.2881 - val_accuracy: 0.8990
Epoch 116/1000
230/230 [==============================] - 0s 88us/step - loss: 0.2077 - accuracy: 0.9435 - val_loss: 0.2826 - val_accuracy: 0.9091
Epoch 117/1000
230/230 [==============================] - 0s 92us/step - loss: 0.2056 - accuracy: 0.9435 - val_loss: 0.2814 - val_accuracy: 0.9091
Epoch 118/1000
230/230 [==============================] - 0s 92us/step - loss: 0.2052 - accuracy: 0.9435 - val_loss: 0.2809 - val_acc

Epoch 168/1000
230/230 [==============================] - 0s 89us/step - loss: 0.1823 - accuracy: 0.9435 - val_loss: 0.2668 - val_accuracy: 0.9091
Epoch 169/1000
230/230 [==============================] - 0s 98us/step - loss: 0.1844 - accuracy: 0.9435 - val_loss: 0.2671 - val_accuracy: 0.9091
Epoch 170/1000
230/230 [==============================] - 0s 92us/step - loss: 0.1815 - accuracy: 0.9435 - val_loss: 0.2667 - val_accuracy: 0.9091
Epoch 171/1000
230/230 [==============================] - 0s 94us/step - loss: 0.1810 - accuracy: 0.9435 - val_loss: 0.2663 - val_accuracy: 0.9091
Epoch 172/1000
230/230 [==============================] - 0s 86us/step - loss: 0.1813 - accuracy: 0.9435 - val_loss: 0.2663 - val_accuracy: 0.9091
Epoch 173/1000
230/230 [==============================] - 0s 89us/step - loss: 0.1806 - accuracy: 0.9435 - val_loss: 0.2662 - val_accuracy: 0.9091
Epoch 174/1000
230/230 [==============================] - 0s 88us/step - loss: 0.1808 - accuracy: 0.9435 - val_loss: 0

230/230 [==============================] - 0s 110us/step - loss: 0.1675 - accuracy: 0.9435 - val_loss: 0.2610 - val_accuracy: 0.9091
Epoch 224/1000
230/230 [==============================] - 0s 122us/step - loss: 0.1680 - accuracy: 0.9435 - val_loss: 0.2569 - val_accuracy: 0.9091
Epoch 225/1000
230/230 [==============================] - 0s 99us/step - loss: 0.1675 - accuracy: 0.9435 - val_loss: 0.2555 - val_accuracy: 0.9091
Epoch 226/1000
230/230 [==============================] - 0s 114us/step - loss: 0.1666 - accuracy: 0.9435 - val_loss: 0.2552 - val_accuracy: 0.9091
Epoch 227/1000
230/230 [==============================] - 0s 93us/step - loss: 0.1666 - accuracy: 0.9435 - val_loss: 0.2557 - val_accuracy: 0.9091
Epoch 228/1000
230/230 [==============================] - 0s 116us/step - loss: 0.1660 - accuracy: 0.9435 - val_loss: 0.2551 - val_accuracy: 0.9091
Epoch 229/1000
230/230 [==============================] - 0s 101us/step - loss: 0.1659 - accuracy: 0.9435 - val_loss: 0.2562 - va

...

230/230 [==============================] - 0s 89us/step - loss: 0.1165 - accuracy: 0.9609 - val_loss: 0.2249 - val_accuracy: 0.9394
Epoch 723/1000
230/230 [==============================] - 0s 100us/step - loss: 0.1161 - accuracy: 0.9609 - val_loss: 0.2295 - val_accuracy: 0.9293
Epoch 724/1000
230/230 [==============================] - 0s 94us/step - loss: 0.1166 - accuracy: 0.9565 - val_loss: 0.2260 - val_accuracy: 0.9394
Epoch 725/1000
230/230 [==============================] - 0s 99us/step - loss: 0.1164 - accuracy: 0.9609 - val_loss: 0.2260 - val_accuracy: 0.9394
Epoch 726/1000
230/230 [==============================] - 0s 98us/step - loss: 0.1162 - accuracy: 0.9609 - val_loss: 0.2312 - val_accuracy: 0.9293
Epoch 727/1000
230/230 [==============================] - 0s 96us/step - loss: 0.1165 - accuracy: 0.9565 - val_loss: 0.2276 - val_accuracy: 0.9394
Epoch 728/1000
230/230 [==============================] - 0s 92us/step - loss: 0.1162 - accuracy: 0.9609 - val_loss: 0.2266 - val_ac

Epoch 778/1000
230/230 [==============================] - 0s 86us/step - loss: 0.1143 - accuracy: 0.9609 - val_loss: 0.2253 - val_accuracy: 0.9394
Epoch 779/1000
230/230 [==============================] - 0s 95us/step - loss: 0.1142 - accuracy: 0.9609 - val_loss: 0.2241 - val_accuracy: 0.9394
Epoch 780/1000
230/230 [==============================] - 0s 103us/step - loss: 0.1143 - accuracy: 0.9609 - val_loss: 0.2258 - val_accuracy: 0.9394
Epoch 781/1000
230/230 [==============================] - 0s 90us/step - loss: 0.1139 - accuracy: 0.9609 - val_loss: 0.2225 - val_accuracy: 0.9394
Epoch 782/1000
230/230 [==============================] - 0s 96us/step - loss: 0.1140 - accuracy: 0.9609 - val_loss: 0.2236 - val_accuracy: 0.9394
Epoch 783/1000
230/230 [==============================] - 0s 94us/step - loss: 0.1139 - accuracy: 0.9609 - val_loss: 0.2316 - val_accuracy: 0.9293
Epoch 784/1000
230/230 [==============================] - 0s 93us/step - loss: 0.1143 - accuracy: 0.9609 - val_loss: 

Epoch 834/1000
230/230 [==============================] - 0s 83us/step - loss: 0.1130 - accuracy: 0.9609 - val_loss: 0.2202 - val_accuracy: 0.9394
Epoch 835/1000
230/230 [==============================] - 0s 97us/step - loss: 0.1126 - accuracy: 0.9609 - val_loss: 0.2294 - val_accuracy: 0.9394
Epoch 836/1000
230/230 [==============================] - 0s 95us/step - loss: 0.1126 - accuracy: 0.9609 - val_loss: 0.2273 - val_accuracy: 0.9394
Epoch 837/1000
230/230 [==============================] - 0s 94us/step - loss: 0.1120 - accuracy: 0.9609 - val_loss: 0.2267 - val_accuracy: 0.9394
Epoch 838/1000
230/230 [==============================] - 0s 94us/step - loss: 0.1120 - accuracy: 0.9609 - val_loss: 0.2280 - val_accuracy: 0.9394
Epoch 839/1000
230/230 [==============================] - 0s 96us/step - loss: 0.1121 - accuracy: 0.9609 - val_loss: 0.2249 - val_accuracy: 0.9394
Epoch 840/1000
230/230 [==============================] - 0s 91us/step - loss: 0.1120 - accuracy: 0.9609 - val_loss: 0

Epoch 890/1000
230/230 [==============================] - 0s 85us/step - loss: 0.1105 - accuracy: 0.9609 - val_loss: 0.2264 - val_accuracy: 0.9394
Epoch 891/1000
230/230 [==============================] - 0s 88us/step - loss: 0.1106 - accuracy: 0.9609 - val_loss: 0.2263 - val_accuracy: 0.9394
Epoch 892/1000
230/230 [==============================] - 0s 103us/step - loss: 0.1104 - accuracy: 0.9609 - val_loss: 0.2259 - val_accuracy: 0.9394
Epoch 893/1000
230/230 [==============================] - 0s 89us/step - loss: 0.1105 - accuracy: 0.9609 - val_loss: 0.2238 - val_accuracy: 0.9394
Epoch 894/1000
230/230 [==============================] - 0s 89us/step - loss: 0.1105 - accuracy: 0.9609 - val_loss: 0.2250 - val_accuracy: 0.9394
Epoch 895/1000
230/230 [==============================] - 0s 87us/step - loss: 0.1105 - accuracy: 0.9609 - val_loss: 0.2317 - val_accuracy: 0.9394
Epoch 896/1000
230/230 [==============================] - 0s 100us/step - loss: 0.1105 - accuracy: 0.9609 - val_loss:

230/230 [==============================] - 0s 86us/step - loss: 0.1094 - accuracy: 0.9609 - val_loss: 0.2269 - val_accuracy: 0.9394
Epoch 946/1000
230/230 [==============================] - 0s 89us/step - loss: 0.1092 - accuracy: 0.9609 - val_loss: 0.2272 - val_accuracy: 0.9394
Epoch 947/1000
230/230 [==============================] - 0s 97us/step - loss: 0.1091 - accuracy: 0.9609 - val_loss: 0.2330 - val_accuracy: 0.9394
Epoch 948/1000
230/230 [==============================] - 0s 90us/step - loss: 0.1092 - accuracy: 0.9609 - val_loss: 0.2308 - val_accuracy: 0.9394
Epoch 949/1000
230/230 [==============================] - 0s 96us/step - loss: 0.1091 - accuracy: 0.9609 - val_loss: 0.2295 - val_accuracy: 0.9394
Epoch 950/1000
230/230 [==============================] - 0s 93us/step - loss: 0.1090 - accuracy: 0.9609 - val_loss: 0.2352 - val_accuracy: 0.9293
Epoch 951/1000
230/230 [==============================] - 0s 98us/step - loss: 0.1093 - accuracy: 0.9609 - val_loss: 0.2298 - val_acc

In [58]:
acc_test_comb2 = model2_comb.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb2*100))


99/99 [==============================] - 0s 130us/step
combination test accuracy: 93.94%


In [63]:
#### improve neural network on combination data
model2_comb2 = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_comb.shape[1],), activity_regularizer=l1(0.001)),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid'),
    Dropout(0.2, ),
])

In [64]:
model2_comb2.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [65]:
model2_comb2.fit(X_sel_train_comb, y_sel_train_comb,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_comb, y_sel_test_comb))

Train on 230 samples, validate on 99 samples
Epoch 1/1000
230/230 [==============================] - 0s 763us/step - loss: 5.6772 - accuracy: 0.4261 - val_loss: 0.8660 - val_accuracy: 0.3939
Epoch 2/1000
230/230 [==============================] - 0s 101us/step - loss: 2.4011 - accuracy: 0.5000 - val_loss: 0.7954 - val_accuracy: 0.5152
Epoch 3/1000
230/230 [==============================] - 0s 138us/step - loss: 1.9242 - accuracy: 0.5435 - val_loss: 0.7677 - val_accuracy: 0.5051
Epoch 4/1000
230/230 [==============================] - 0s 139us/step - loss: 2.5729 - accuracy: 0.4826 - val_loss: 0.7577 - val_accuracy: 0.5455
Epoch 5/1000
230/230 [==============================] - 0s 153us/step - loss: 2.2730 - accuracy: 0.5261 - val_loss: 0.7513 - val_accuracy: 0.5253
Epoch 6/1000
230/230 [==============================] - 0s 129us/step - loss: 2.3338 - accuracy: 0.5130 - val_loss: 0.7443 - val_accuracy: 0.5354
Epoch 7/1000
230/230 [==============================] - 0s 136us/step - loss: 2

230/230 [==============================] - 0s 143us/step - loss: 1.7374 - accuracy: 0.8130 - val_loss: 0.5473 - val_accuracy: 0.7879
Epoch 57/1000
230/230 [==============================] - 0s 109us/step - loss: 2.3238 - accuracy: 0.7739 - val_loss: 0.5460 - val_accuracy: 0.7879
Epoch 58/1000
230/230 [==============================] - 0s 118us/step - loss: 2.1944 - accuracy: 0.7739 - val_loss: 0.5499 - val_accuracy: 0.7879
Epoch 59/1000
230/230 [==============================] - 0s 119us/step - loss: 1.8534 - accuracy: 0.7913 - val_loss: 0.5361 - val_accuracy: 0.7879
Epoch 60/1000
230/230 [==============================] - 0s 109us/step - loss: 1.8591 - accuracy: 0.8130 - val_loss: 0.5296 - val_accuracy: 0.7879
Epoch 61/1000
230/230 [==============================] - 0s 100us/step - loss: 1.9597 - accuracy: 0.8217 - val_loss: 0.5272 - val_accuracy: 0.7879
Epoch 62/1000
230/230 [==============================] - 0s 108us/step - loss: 2.3176 - accuracy: 0.7739 - val_loss: 0.5308 - val_ac

Epoch 112/1000
230/230 [==============================] - 0s 85us/step - loss: 1.7129 - accuracy: 0.8435 - val_loss: 0.4140 - val_accuracy: 0.9192
Epoch 113/1000
230/230 [==============================] - 0s 101us/step - loss: 1.5250 - accuracy: 0.8522 - val_loss: 0.4154 - val_accuracy: 0.9192
Epoch 114/1000
230/230 [==============================] - 0s 108us/step - loss: 1.8358 - accuracy: 0.8391 - val_loss: 0.4142 - val_accuracy: 0.9192
Epoch 115/1000
230/230 [==============================] - 0s 103us/step - loss: 2.0238 - accuracy: 0.8304 - val_loss: 0.4126 - val_accuracy: 0.9192
Epoch 116/1000
230/230 [==============================] - 0s 109us/step - loss: 1.6350 - accuracy: 0.8565 - val_loss: 0.4110 - val_accuracy: 0.9192
Epoch 117/1000
230/230 [==============================] - 0s 104us/step - loss: 1.8684 - accuracy: 0.8130 - val_loss: 0.4071 - val_accuracy: 0.8889
Epoch 118/1000
230/230 [==============================] - 0s 112us/step - loss: 1.6399 - accuracy: 0.8478 - val_l

230/230 [==============================] - 0s 93us/step - loss: 1.4773 - accuracy: 0.8522 - val_loss: 0.3563 - val_accuracy: 0.8889
Epoch 168/1000
230/230 [==============================] - 0s 99us/step - loss: 1.5374 - accuracy: 0.8565 - val_loss: 0.3574 - val_accuracy: 0.8889
Epoch 169/1000
230/230 [==============================] - 0s 116us/step - loss: 2.0706 - accuracy: 0.8217 - val_loss: 0.3582 - val_accuracy: 0.8889
Epoch 170/1000
230/230 [==============================] - 0s 109us/step - loss: 1.5981 - accuracy: 0.8522 - val_loss: 0.3761 - val_accuracy: 0.8889
Epoch 171/1000
230/230 [==============================] - 0s 109us/step - loss: 1.4773 - accuracy: 0.8522 - val_loss: 0.3650 - val_accuracy: 0.8889
Epoch 172/1000
230/230 [==============================] - 0s 104us/step - loss: 1.7261 - accuracy: 0.8391 - val_loss: 0.3573 - val_accuracy: 0.8889
Epoch 173/1000
230/230 [==============================] - 0s 112us/step - loss: 1.4663 - accuracy: 0.8609 - val_loss: 0.3681 - va

230/230 [==============================] - 0s 107us/step - loss: 1.3753 - accuracy: 0.8696 - val_loss: 0.3227 - val_accuracy: 0.8990
Epoch 223/1000
230/230 [==============================] - 0s 108us/step - loss: 1.9668 - accuracy: 0.8391 - val_loss: 0.3347 - val_accuracy: 0.9293
Epoch 224/1000
230/230 [==============================] - 0s 117us/step - loss: 2.3651 - accuracy: 0.8130 - val_loss: 0.3264 - val_accuracy: 0.9293
Epoch 225/1000
230/230 [==============================] - 0s 125us/step - loss: 1.5965 - accuracy: 0.8870 - val_loss: 0.3318 - val_accuracy: 0.9293
Epoch 226/1000
230/230 [==============================] - 0s 117us/step - loss: 1.0328 - accuracy: 0.9000 - val_loss: 0.3332 - val_accuracy: 0.9293
Epoch 227/1000
230/230 [==============================] - 0s 106us/step - loss: 1.3755 - accuracy: 0.8696 - val_loss: 0.3420 - val_accuracy: 0.9293
Epoch 228/1000
230/230 [==============================] - 0s 121us/step - loss: 1.3820 - accuracy: 0.8696 - val_loss: 0.3506 - 

...

230/230 [==============================] - 0s 115us/step - loss: 1.8100 - accuracy: 0.8522 - val_loss: 0.2995 - val_accuracy: 0.9394
Epoch 773/1000
230/230 [==============================] - 0s 132us/step - loss: 1.6646 - accuracy: 0.8652 - val_loss: 0.2975 - val_accuracy: 0.9394
Epoch 774/1000
230/230 [==============================] - 0s 129us/step - loss: 1.6717 - accuracy: 0.8609 - val_loss: 0.3002 - val_accuracy: 0.9394
Epoch 775/1000
230/230 [==============================] - 0s 128us/step - loss: 1.2747 - accuracy: 0.8870 - val_loss: 0.2952 - val_accuracy: 0.9394
Epoch 776/1000
230/230 [==============================] - 0s 162us/step - loss: 1.7473 - accuracy: 0.8522 - val_loss: 0.2919 - val_accuracy: 0.9394
Epoch 777/1000
230/230 [==============================] - 0s 234us/step - loss: 1.4862 - accuracy: 0.8696 - val_loss: 0.3026 - val_accuracy: 0.9394
Epoch 778/1000
230/230 [==============================] - 0s 219us/step - loss: 1.1456 - accuracy: 0.8913 - val_loss: 0.3036 - 

230/230 [==============================] - 0s 98us/step - loss: 1.3886 - accuracy: 0.8870 - val_loss: 0.2925 - val_accuracy: 0.9394
Epoch 828/1000
230/230 [==============================] - 0s 105us/step - loss: 1.7445 - accuracy: 0.8565 - val_loss: 0.2994 - val_accuracy: 0.9394
Epoch 829/1000
230/230 [==============================] - 0s 118us/step - loss: 1.8028 - accuracy: 0.8522 - val_loss: 0.3085 - val_accuracy: 0.9394
Epoch 830/1000
230/230 [==============================] - 0s 102us/step - loss: 1.2574 - accuracy: 0.8957 - val_loss: 0.3036 - val_accuracy: 0.9394
Epoch 831/1000
230/230 [==============================] - 0s 106us/step - loss: 1.2614 - accuracy: 0.8913 - val_loss: 0.3007 - val_accuracy: 0.9394
Epoch 832/1000
230/230 [==============================] - 0s 124us/step - loss: 1.1287 - accuracy: 0.9000 - val_loss: 0.3001 - val_accuracy: 0.9394
Epoch 833/1000
230/230 [==============================] - 0s 96us/step - loss: 1.6010 - accuracy: 0.8652 - val_loss: 0.3019 - va

230/230 [==============================] - 0s 106us/step - loss: 1.4740 - accuracy: 0.8696 - val_loss: 0.2991 - val_accuracy: 0.9394
Epoch 883/1000
230/230 [==============================] - 0s 116us/step - loss: 1.1385 - accuracy: 0.8913 - val_loss: 0.3049 - val_accuracy: 0.9394
Epoch 884/1000
230/230 [==============================] - 0s 125us/step - loss: 1.9827 - accuracy: 0.8522 - val_loss: 0.2951 - val_accuracy: 0.9394
Epoch 885/1000
230/230 [==============================] - 0s 100us/step - loss: 1.8469 - accuracy: 0.8609 - val_loss: 0.2966 - val_accuracy: 0.9394
Epoch 886/1000
230/230 [==============================] - 0s 115us/step - loss: 1.6612 - accuracy: 0.8652 - val_loss: 0.2952 - val_accuracy: 0.9394
Epoch 887/1000
230/230 [==============================] - 0s 114us/step - loss: 1.5969 - accuracy: 0.8652 - val_loss: 0.2955 - val_accuracy: 0.9394
Epoch 888/1000
230/230 [==============================] - 0s 111us/step - loss: 1.7852 - accuracy: 0.8609 - val_loss: 0.2913 - 

230/230 [==============================] - 0s 100us/step - loss: 1.7645 - accuracy: 0.8565 - val_loss: 0.3013 - val_accuracy: 0.9394
Epoch 938/1000
230/230 [==============================] - 0s 124us/step - loss: 2.0094 - accuracy: 0.8478 - val_loss: 0.3001 - val_accuracy: 0.9394
Epoch 939/1000
230/230 [==============================] - 0s 145us/step - loss: 1.1654 - accuracy: 0.8957 - val_loss: 0.2992 - val_accuracy: 0.9394
Epoch 940/1000
230/230 [==============================] - 0s 134us/step - loss: 1.4961 - accuracy: 0.8739 - val_loss: 0.3025 - val_accuracy: 0.9394
Epoch 941/1000
230/230 [==============================] - 0s 136us/step - loss: 1.6321 - accuracy: 0.8652 - val_loss: 0.2987 - val_accuracy: 0.9394
Epoch 942/1000
230/230 [==============================] - 0s 128us/step - loss: 1.2290 - accuracy: 0.8913 - val_loss: 0.3010 - val_accuracy: 0.9394
Epoch 943/1000
230/230 [==============================] - 0s 131us/step - loss: 1.7649 - accuracy: 0.8565 - val_loss: 0.3025 - 

230/230 [==============================] - 0s 102us/step - loss: 1.6658 - accuracy: 0.8739 - val_loss: 0.3069 - val_accuracy: 0.9394
Epoch 993/1000
230/230 [==============================] - 0s 102us/step - loss: 1.2962 - accuracy: 0.8870 - val_loss: 0.3063 - val_accuracy: 0.9394
Epoch 994/1000
230/230 [==============================] - 0s 114us/step - loss: 1.5419 - accuracy: 0.8783 - val_loss: 0.3068 - val_accuracy: 0.9394
Epoch 995/1000
230/230 [==============================] - 0s 106us/step - loss: 1.6302 - accuracy: 0.8652 - val_loss: 0.3013 - val_accuracy: 0.9394
Epoch 996/1000
230/230 [==============================] - 0s 102us/step - loss: 1.4973 - accuracy: 0.8739 - val_loss: 0.3086 - val_accuracy: 0.9394
Epoch 997/1000
230/230 [==============================] - 0s 105us/step - loss: 1.6963 - accuracy: 0.8609 - val_loss: 0.3082 - val_accuracy: 0.9394
Epoch 998/1000
230/230 [==============================] - 0s 104us/step - loss: 1.6867 - accuracy: 0.8652 - val_loss: 0.3108 - 

In [66]:
acc_test_comb3 = model2_comb2.evaluate(X_sel_test_comb, y_sel_test_comb)[1]
print('combination test accuracy: %.2f%%' % (acc_test_comb3*100))

99/99 [==============================] - 0s 136us/step
combination test accuracy: 93.94%


In [67]:
#### neural network on over-sampling data
model2_over = Sequential([
    Dense(32, activation='relu', input_shape=(X_sel_train_over.shape[1],)),
    Dense(1, activation='sigmoid'),
])

In [68]:
model2_over.compile(optimizer='sgd',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [69]:
model2_over.fit(X_sel_train_over, y_sel_train_over,
          batch_size=32, epochs=1000,
          validation_data=(X_sel_test_over, y_sel_test_over))

Train on 305 samples, validate on 131 samples
Epoch 1/1000
305/305 [==============================] - 0s 358us/step - loss: 44.1890 - accuracy: 0.4754 - val_loss: 0.7780 - val_accuracy: 0.4580
Epoch 2/1000
305/305 [==============================] - 0s 79us/step - loss: 0.7833 - accuracy: 0.4459 - val_loss: 0.7633 - val_accuracy: 0.4580
Epoch 3/1000
305/305 [==============================] - 0s 106us/step - loss: 0.7639 - accuracy: 0.4459 - val_loss: 0.7458 - val_accuracy: 0.4580
Epoch 4/1000
305/305 [==============================] - 0s 106us/step - loss: 0.7470 - accuracy: 0.4459 - val_loss: 0.7307 - val_accuracy: 0.4580
Epoch 5/1000
305/305 [==============================] - 0s 88us/step - loss: 0.7306 - accuracy: 0.4459 - val_loss: 0.7183 - val_accuracy: 0.4580
Epoch 6/1000
305/305 [==============================] - 0s 110us/step - loss: 0.7190 - accuracy: 0.4426 - val_loss: 0.7098 - val_accuracy: 0.4580
Epoch 7/1000
305/305 [==============================] - 0s 110us/step - loss: 0

305/305 [==============================] - 0s 113us/step - loss: 0.5885 - accuracy: 0.6623 - val_loss: 0.5527 - val_accuracy: 0.7481
Epoch 57/1000
305/305 [==============================] - 0s 122us/step - loss: 0.5650 - accuracy: 0.7049 - val_loss: 0.5496 - val_accuracy: 0.7481
Epoch 58/1000
305/305 [==============================] - 0s 96us/step - loss: 0.5584 - accuracy: 0.7246 - val_loss: 0.5453 - val_accuracy: 0.7405
Epoch 59/1000
305/305 [==============================] - 0s 81us/step - loss: 0.5653 - accuracy: 0.7246 - val_loss: 0.5509 - val_accuracy: 0.7481
Epoch 60/1000
305/305 [==============================] - 0s 80us/step - loss: 0.5594 - accuracy: 0.7115 - val_loss: 0.5640 - val_accuracy: 0.6870
Epoch 61/1000
305/305 [==============================] - 0s 92us/step - loss: 0.5637 - accuracy: 0.7148 - val_loss: 0.5457 - val_accuracy: 0.7481
Epoch 62/1000
305/305 [==============================] - 0s 88us/step - loss: 0.5515 - accuracy: 0.7246 - val_loss: 0.5374 - val_accurac

Epoch 112/1000
305/305 [==============================] - 0s 77us/step - loss: 0.4694 - accuracy: 0.7541 - val_loss: 0.4939 - val_accuracy: 0.7634
Epoch 113/1000
305/305 [==============================] - 0s 105us/step - loss: 0.4525 - accuracy: 0.7607 - val_loss: 0.4876 - val_accuracy: 0.7634
Epoch 114/1000
305/305 [==============================] - 0s 119us/step - loss: 0.5041 - accuracy: 0.7443 - val_loss: 0.5008 - val_accuracy: 0.7634
Epoch 115/1000
305/305 [==============================] - 0s 140us/step - loss: 0.4537 - accuracy: 0.7607 - val_loss: 0.5059 - val_accuracy: 0.7634
Epoch 116/1000
305/305 [==============================] - 0s 104us/step - loss: 0.4621 - accuracy: 0.7541 - val_loss: 0.4557 - val_accuracy: 0.7557
Epoch 117/1000
305/305 [==============================] - 0s 101us/step - loss: 0.5461 - accuracy: 0.7377 - val_loss: 0.5210 - val_accuracy: 0.7710
Epoch 118/1000
305/305 [==============================] - 0s 99us/step - loss: 0.5020 - accuracy: 0.7508 - val_lo

305/305 [==============================] - 0s 76us/step - loss: 0.4613 - accuracy: 0.8262 - val_loss: 0.4491 - val_accuracy: 0.7710
Epoch 168/1000
305/305 [==============================] - 0s 99us/step - loss: 0.4986 - accuracy: 0.8000 - val_loss: 0.4720 - val_accuracy: 0.8244
Epoch 169/1000
305/305 [==============================] - 0s 87us/step - loss: 0.4630 - accuracy: 0.8295 - val_loss: 0.4450 - val_accuracy: 0.7939
Epoch 170/1000
305/305 [==============================] - 0s 85us/step - loss: 0.4383 - accuracy: 0.8295 - val_loss: 0.4298 - val_accuracy: 0.8168
Epoch 171/1000
305/305 [==============================] - 0s 84us/step - loss: 0.4162 - accuracy: 0.8393 - val_loss: 0.4842 - val_accuracy: 0.7252
Epoch 172/1000
305/305 [==============================] - 0s 92us/step - loss: 0.4650 - accuracy: 0.8066 - val_loss: 0.4660 - val_accuracy: 0.7634
Epoch 173/1000
305/305 [==============================] - 0s 94us/step - loss: 0.4745 - accuracy: 0.8230 - val_loss: 0.4934 - val_acc

305/305 [==============================] - 0s 87us/step - loss: 0.4406 - accuracy: 0.8262 - val_loss: 0.5072 - val_accuracy: 0.8321
Epoch 223/1000
305/305 [==============================] - 0s 96us/step - loss: 0.4437 - accuracy: 0.8295 - val_loss: 0.4597 - val_accuracy: 0.8244
Epoch 224/1000
305/305 [==============================] - 0s 86us/step - loss: 0.4774 - accuracy: 0.8197 - val_loss: 0.4540 - val_accuracy: 0.8244
Epoch 225/1000
305/305 [==============================] - 0s 85us/step - loss: 0.4326 - accuracy: 0.8230 - val_loss: 0.4656 - val_accuracy: 0.8321
Epoch 226/1000
305/305 [==============================] - 0s 93us/step - loss: 0.4524 - accuracy: 0.8230 - val_loss: 0.4500 - val_accuracy: 0.7557
Epoch 227/1000
305/305 [==============================] - 0s 82us/step - loss: 0.4438 - accuracy: 0.8328 - val_loss: 0.6164 - val_accuracy: 0.6718
Epoch 228/1000
305/305 [==============================] - 0s 85us/step - loss: 0.4713 - accuracy: 0.8098 - val_loss: 0.4360 - val_acc

...

Epoch 723/1000
305/305 [==============================] - 0s 112us/step - loss: 0.3267 - accuracy: 0.8557 - val_loss: 0.3759 - val_accuracy: 0.8244
Epoch 724/1000
305/305 [==============================] - 0s 76us/step - loss: 0.3205 - accuracy: 0.8623 - val_loss: 0.3484 - val_accuracy: 0.8321
Epoch 725/1000
305/305 [==============================] - 0s 79us/step - loss: 0.3324 - accuracy: 0.8557 - val_loss: 0.3480 - val_accuracy: 0.8473
Epoch 726/1000
305/305 [==============================] - 0s 91us/step - loss: 0.3330 - accuracy: 0.8590 - val_loss: 0.3520 - val_accuracy: 0.8473
Epoch 727/1000
305/305 [==============================] - 0s 79us/step - loss: 0.3277 - accuracy: 0.8623 - val_loss: 0.3488 - val_accuracy: 0.8321
Epoch 728/1000
305/305 [==============================] - 0s 82us/step - loss: 0.3343 - accuracy: 0.8492 - val_loss: 0.3443 - val_accuracy: 0.8473
Epoch 729/1000
305/305 [==============================] - 0s 79us/step - loss: 0.3311 - accuracy: 0.8590 - val_loss: 

Epoch 779/1000
305/305 [==============================] - 0s 81us/step - loss: 0.3231 - accuracy: 0.8590 - val_loss: 0.3543 - val_accuracy: 0.8321
Epoch 780/1000
305/305 [==============================] - 0s 90us/step - loss: 0.3175 - accuracy: 0.8590 - val_loss: 0.3513 - val_accuracy: 0.8321
Epoch 781/1000
305/305 [==============================] - 0s 85us/step - loss: 0.3256 - accuracy: 0.8459 - val_loss: 0.3609 - val_accuracy: 0.8321
Epoch 782/1000
305/305 [==============================] - 0s 84us/step - loss: 0.3331 - accuracy: 0.8525 - val_loss: 0.3677 - val_accuracy: 0.8321
Epoch 783/1000
305/305 [==============================] - 0s 101us/step - loss: 0.3204 - accuracy: 0.8623 - val_loss: 0.3609 - val_accuracy: 0.8321
Epoch 784/1000
305/305 [==============================] - 0s 92us/step - loss: 0.3164 - accuracy: 0.8656 - val_loss: 0.3461 - val_accuracy: 0.8473
Epoch 785/1000
305/305 [==============================] - 0s 81us/step - loss: 0.3174 - accuracy: 0.8590 - val_loss: 

Epoch 835/1000
305/305 [==============================] - 0s 80us/step - loss: 0.3221 - accuracy: 0.8590 - val_loss: 0.3405 - val_accuracy: 0.8473
Epoch 836/1000
305/305 [==============================] - 0s 93us/step - loss: 0.3205 - accuracy: 0.8623 - val_loss: 0.3381 - val_accuracy: 0.8473
Epoch 837/1000
305/305 [==============================] - 0s 82us/step - loss: 0.3158 - accuracy: 0.8590 - val_loss: 0.3390 - val_accuracy: 0.8473
Epoch 838/1000
305/305 [==============================] - 0s 85us/step - loss: 0.3270 - accuracy: 0.8557 - val_loss: 0.3613 - val_accuracy: 0.8321
Epoch 839/1000
305/305 [==============================] - 0s 85us/step - loss: 0.3246 - accuracy: 0.8623 - val_loss: 0.3516 - val_accuracy: 0.8321
Epoch 840/1000
305/305 [==============================] - 0s 84us/step - loss: 0.3190 - accuracy: 0.8656 - val_loss: 0.3372 - val_accuracy: 0.8473
Epoch 841/1000
305/305 [==============================] - 0s 89us/step - loss: 0.3136 - accuracy: 0.8689 - val_loss: 0

305/305 [==============================] - 0s 82us/step - loss: 0.3072 - accuracy: 0.8623 - val_loss: 0.3415 - val_accuracy: 0.8321
Epoch 891/1000
305/305 [==============================] - 0s 88us/step - loss: 0.3079 - accuracy: 0.8623 - val_loss: 0.3478 - val_accuracy: 0.8321
Epoch 892/1000
305/305 [==============================] - 0s 82us/step - loss: 0.3097 - accuracy: 0.8525 - val_loss: 0.3390 - val_accuracy: 0.8321
Epoch 893/1000
305/305 [==============================] - 0s 89us/step - loss: 0.3078 - accuracy: 0.8623 - val_loss: 0.3385 - val_accuracy: 0.8321
Epoch 894/1000
305/305 [==============================] - 0s 80us/step - loss: 0.3088 - accuracy: 0.8623 - val_loss: 0.3355 - val_accuracy: 0.8321
Epoch 895/1000
305/305 [==============================] - 0s 83us/step - loss: 0.3081 - accuracy: 0.8557 - val_loss: 0.3502 - val_accuracy: 0.8321
Epoch 896/1000
305/305 [==============================] - 0s 89us/step - loss: 0.3068 - accuracy: 0.8557 - val_loss: 0.3379 - val_acc

Epoch 946/1000
305/305 [==============================] - 0s 85us/step - loss: 0.3034 - accuracy: 0.8623 - val_loss: 0.3333 - val_accuracy: 0.8321
Epoch 947/1000
305/305 [==============================] - 0s 91us/step - loss: 0.3028 - accuracy: 0.8623 - val_loss: 0.3395 - val_accuracy: 0.8321
Epoch 948/1000
305/305 [==============================] - 0s 79us/step - loss: 0.3029 - accuracy: 0.8623 - val_loss: 0.3398 - val_accuracy: 0.8321
Epoch 949/1000
305/305 [==============================] - 0s 90us/step - loss: 0.3054 - accuracy: 0.8623 - val_loss: 0.3416 - val_accuracy: 0.8321
Epoch 950/1000
305/305 [==============================] - 0s 87us/step - loss: 0.3034 - accuracy: 0.8590 - val_loss: 0.3379 - val_accuracy: 0.8321
Epoch 951/1000
305/305 [==============================] - 0s 84us/step - loss: 0.3014 - accuracy: 0.8623 - val_loss: 0.3375 - val_accuracy: 0.8321
Epoch 952/1000
305/305 [==============================] - 0s 94us/step - loss: 0.3014 - accuracy: 0.8623 - val_loss: 0

In [70]:
acc_test_over2 = model2_over.evaluate(X_sel_test_over, y_sel_test_over)[1]
print('combination test accuracy: %.2f%%' % (acc_test_over2*100))

131/131 [==============================] - 0s 74us/step
combination test accuracy: 83.21%


In [76]:
############ Logistic Regression ############

In [77]:
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [78]:
###### logistics on combination data
log_comb = LogisticRegression()
log_comb.fit(X_train_comb, y_train_comb)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [79]:
y_pred_comb = log_comb.predict(X_test_comb)
print('combination logistic test accuracy %.2f%%' % (log_comb.score(X_test_comb, y_test_comb)*100))

combination logistic test accuracy 97.03%


In [80]:
##### logistics on over-sampling data
log_over = LogisticRegression()
log_over.fit(X_train_over, y_train_over)

/Users/Rebecca/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
                   intercept_scaling=1, l1_ratio=None, max_iter=100,
                   multi_class='warn', n_jobs=None, penalty='l2',
                   random_state=None, solver='warn', tol=0.0001, verbose=0,
                   warm_start=False)

In [81]:
y_pred_over = log_over.predict(X_test_over)
print('over-sampling logistic test accuracy %.2f%%' % (log_over.score(X_test_over, y_test_over)*100))

over-sampling logistic test accuracy 90.84%


In [82]:
############## Random Forest ##############

In [83]:
###### random forest on combination data
from sklearn.ensemble import RandomForestClassifier

rf_comb = RandomForestClassifier(n_estimators=100)
rf_comb.fit(X_train_comb,y_train_comb)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [84]:
from sklearn.metrics import accuracy_score
y_pred_comb = rf_comb.predict(X_test_comb)
print('combination test accuracy: %.2f%%' % (accuracy_score(y_test_comb, y_pred_comb)*100))

combination test accuracy: 98.02%


In [85]:
###### random forest on over-sampling data
from sklearn.ensemble import RandomForestClassifier

rf_over = RandomForestClassifier(n_estimators=100)
rf_over.fit(X_train_over,y_train_over)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)

In [86]:
from sklearn.metrics import accuracy_score
y_pred_over = rf_over.predict(X_test_over)
print('over-sampling test accuracy: %.2f%%' % (accuracy_score(y_test_over, y_pred_over)*100))

over-sampling test accuracy: 96.95%


In [87]:
#### Random forest with cross-validation
## Retrieved from https://stackabuse.com/cross-validation-and-grid-search-for-model-selection-in-python/

In [88]:
## random forest model with CV on combination data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_comb = RandomForestClassifier(n_estimators=100, random_state=123)

accs_comb = cross_val_score(estimator=rfcv_comb, X=X_train_comb, y=y_train_comb, cv=5)
print(accs_comb)
print(accs_comb.mean())

[1.         1.         0.9787234  0.95652174 0.97826087]
0.9827012025901943


In [89]:
## random forest model with CV on over-sampling data
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

rfcv_over = RandomForestClassifier(n_estimators=100, random_state=123)

accs_over = cross_val_score(estimator=rfcv_over, X=X_train_over, y=y_train_over, cv=5)
print(accs_over)
print(accs_over.mean())

[0.93548387 0.98387097 0.8852459  0.96666667 0.96666667]
0.9475868147364711
